In [2]:
# Import all the necessary modules
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as mtick
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
import pandas_datareader as pdr
import math
import datetime
import itertools
import yfinance as yf
import seaborn as sn
from IPython.display import display, HTML
from trend_following_signal import (apply_jupyter_fullscreen_css, load_financial_data, get_returns_volatility, calculate_slope, trend_signal, slope_signal, 
                             create_trend_strategy, get_close_prices, calculate_donchian_channels)
from strategy_performance import (calculate_sharpe_ratio, calculate_calmar_ratio, calculate_CAGR, calculate_risk_and_performance_metrics,
                                          calculate_compounded_cumulative_returns, estimate_fee_per_trade, rolling_sharpe_ratio)
import coinbase_utils as cn
import strategy_performance as perf
import position_sizing_binary_utils as size_bin
import position_sizing_continuous_utils as size_cont
import trend_following_signal as tf
%matplotlib inline

In [4]:
import importlib
importlib.reload(cn)
importlib.reload(perf)
importlib.reload(tf)
importlib.reload(size_bin)
importlib.reload(size_cont)

<module 'position_sizing_continuous_utils' from '/Users/adheerchauhan/Documents/git/trend_following/position_sizing_continuous_utils.py'>

In [6]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('Display.max_rows', None)
pd.set_option('Display.max_columns',None)
apply_jupyter_fullscreen_css()

In [5]:
from strategy_performance import calculate_risk_and_performance_metrics

import seaborn as sns

def plot_moving_avg_crossover_performance_heatmap(df_performance):
    unique_step_sizes = df_performance['stepsize'].unique()

    # Plotting each heatmap in a loop
    for step in unique_step_sizes:
        subset = df_performance[df_performance['stepsize'] == step]
        pivoted_df_sharpe = subset.pivot(index='slow_mavg', columns='fast_mavg', values='sharpe_ratio')
        pivoted_df_calmar = subset.pivot(index='slow_mavg', columns='fast_mavg', values='calmar_ratio')
        pivoted_df_return = subset.pivot(index='slow_mavg', columns='fast_mavg', values='annualized_return')
        
        fig = plt.figure(figsize=(30,6))
        # plt.style.use('bmh')
        layout = (1,3)
        sharpe_ax = plt.subplot2grid(layout, (0,0))#, colspan=2)
        calmar_ax = plt.subplot2grid(layout, (0,1))#, colspan=2)
        return_ax = plt.subplot2grid(layout, (0,2))#, colspan=2)

        sns.heatmap(pivoted_df_sharpe, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=sharpe_ax)
        _ = sharpe_ax.set_title(f'Sharpe Ratio Heatmap\nStep Size: {step}')
        _ = sharpe_ax.set_ylabel('Slow Moving Average (Days)')
        _ = sharpe_ax.set_xlabel('Fast Moving Average (Days)')

        sns.heatmap(pivoted_df_calmar, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=calmar_ax)
        _ = calmar_ax.set_title(f'Calmar Ratio Heatmap\nStep Size: {step}')
        _ = calmar_ax.set_ylabel('Slow Moving Average (Days)')
        _ = calmar_ax.set_xlabel('Fast Moving Average (Days)')

        sns.heatmap(pivoted_df_return, annot=True, fmt=".2f", cmap='RdYlGn', linewidths=.5, ax=return_ax)
        _ = return_ax.set_title(f'Annualized Return Heatmap\nStep Size: {step}')
        _ = return_ax.set_ylabel('Slow Moving Average (Days)')
        _ = return_ax.set_xlabel('Fast Moving Average (Days)')

        plt.tight_layout()
    
    return

def moving_avg_crossover_strategy_performance(start_date, end_date, ticker_list, fast_mavg_list=np.arange(10, 101, 10), slow_mavg_list=np.arange(50, 501, 50), rolling_donchian_window=20, long_only=True,
                                              initial_capital=15000, rolling_cov_window=20, volatility_window=20, transaction_cost_est=0.001, passive_trade_rate=0.05, use_coinbase_data=True,
                                              rolling_sharpe_window=50, cash_buffer_percentage=0.10, annualized_target_volatility=0.20, annual_trading_days=365, use_specific_start_date=False,
                                              signal_start_date=None):
    
    perf_cols = ['fast_mavg', 'slow_mavg', 'stepsize', 'annualized_return', 'sharpe_ratio', 'calmar_ratio', 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration',
                'hit_rate', 't_statistic', 'p_value', 'trade_count']
    df_performance = pd.DataFrame(columns=perf_cols)
    
    mavg_stepsize_list = [2, 4, 6, 8]
    for slow_mavg in slow_mavg_list:
        for fast_mavg in fast_mavg_list:
            for stepsize in mavg_stepsize_list:
                if fast_mavg < slow_mavg:
                    df = size.apply_target_volatility_position_sizing_strategy(start_date, end_date, ticker_list, fast_mavg, slow_mavg,
                                                                               mavg_stepsize, rolling_donchian_window, long_only, initial_capital,
                                                                               rolling_cov_window, volatility_window, transaction_cost_est, passive_trade_rate,
                                                                               use_coinbase_data, rolling_sharpe_window, cash_buffer_percentage, annualized_target_volatility,
                                                                               annual_trading_days, use_specific_start_date, signal_start_date)
                    performance_metrics = calculate_risk_and_performance_metrics(df, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                                strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                                passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)
                    row = {
                        # 'ticker': ticker,
                        'fast_mavg': fast_mavg,
                        'slow_mavg': slow_mavg,
                        'stepsize': stepsize,
                        'annualized_return': performance_metrics['annualized_return'],
                        'sharpe_ratio': performance_metrics['annualized_sharpe_ratio'],
                        'calmar_ratio': performance_metrics['calmar_ratio'],
                        'annualized_std_dev': performance_metrics['annualized_std_dev'],
                        'max_drawdown': performance_metrics['max_drawdown'],
                        'max_drawdown_duration': performance_metrics['max_drawdown_duration'],
                        'hit_rate': performance_metrics['hit_rate'],
                        't_statistic': performance_metrics['t_statistic'],
                        'p_value': performance_metrics['p_value'],
                        'trade_count': performance_metrics['trade_count']
                    }
                    df_performance = pd.concat([df_performance, pd.DataFrame([row])], ignore_index=True)
    
    return df_performance

In [6]:
def plot_trend_following_performance_time_series(df, df_trend, start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, price_or_returns_calc, rolling_donchian_window, rolling_sharpe_window):
    
    start_date = pd.to_datetime(start_date).date().strftime('%Y-%m-%d')
    end_date = pd.to_datetime(end_date).date().strftime('%Y-%m-%d')
    fig = plt.figure(figsize=(22,20))
    layout = (5,2)
    trend_ax = plt.subplot2grid(layout, (0,0), colspan=2)
    trend_donchian_ax = plt.subplot2grid(layout, (1,0), colspan=2)
    trend_signal_ax = plt.subplot2grid(layout, (2,0), colspan=2)
    trend_signal_ax2 = trend_signal_ax.twinx()
    trend_rolling_sharpe_ax = plt.subplot2grid(layout, (3,0), colspan=2)
    trend_return_ax = plt.subplot2grid(layout, (4,0))#, colspan=2)
    trend_cum_return_ax = plt.subplot2grid(layout, (4,1))#, colspan=2)

    _ = trend_ax.plot(df.index, df[f'{ticker}_t_1_close'], label='Price')
    for mavg in np.linspace(fast_mavg, slow_mavg, mavg_stepsize):
        _ = trend_ax.plot(df_trend.index, df_trend[f'{ticker}_{int(mavg)}_mavg'], label=f'{mavg} M Avg')

    _ = trend_ax.set_title(f'{ticker} Moving Average Ribbons - {start_date} - {end_date}')
    _ = trend_ax.set_ylabel('Price')
    _ = trend_ax.set_xlabel('Date')
    _ = trend_ax.legend(loc='upper left')
    _ = trend_ax.grid()

    _ = trend_donchian_ax.plot(df.index, df[f'{ticker}_t_1_close'], label='Price')
    _ = trend_donchian_ax.plot(df.index,
                      df[f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}_t_2'], label='Donchian Upper Band', linestyle='--', linewidth=3)
    _ = trend_donchian_ax.plot(df.index,
                      df[f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}_t_2'], label='Donchian Lower Band', linestyle='--', linewidth=3)

    _ = trend_donchian_ax.set_title(f'{ticker} Donchian Channels')
    _ = trend_donchian_ax.set_ylabel('Price')
    _ = trend_donchian_ax.set_xlabel('Date')
    _ = trend_donchian_ax.legend(loc='upper left')
    _ = trend_donchian_ax.grid()

    _ = trend_signal_ax.plot(df.index,
                             df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_signal'], label='Signal')
    _ = trend_signal_ax2.plot(df.index, df[f'{ticker}_t_1_close'], label='Price', alpha=0.8, color='orange', linestyle='--')
    _ = trend_signal_ax.set_title(f'{ticker} Trend Strategy with Donchian Channel Signal')
    _ = trend_signal_ax.set_ylabel('Signal')
    _ = trend_signal_ax.set_xlabel('Date')
    _ = trend_signal_ax.legend(loc='upper left')
    _ = trend_signal_ax.grid()

    _ = trend_rolling_sharpe_ax.plot(df.index, df[f'portfolio_rolling_sharpe_50'],
                                     # df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_rolling_sharpe_{rolling_sharpe_window}'],
                                     label='Sharpe Ratio', color='orange')
    _ = trend_rolling_sharpe_ax.set_title(f'{ticker} Rolling Sharpe Ratio')
    _ = trend_rolling_sharpe_ax.set_ylabel('Sharpe Ratio')
    _ = trend_rolling_sharpe_ax.set_xlabel('Date')
    _ = trend_rolling_sharpe_ax.legend(loc='upper left')
    _ = trend_rolling_sharpe_ax.grid()

    _ = trend_return_ax.plot(df.index, df[f'portfolio_daily_pct_returns'],
                             # df[f'{ticker}_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_mavg_crossover_{rolling_donchian_window}_donchian_strategy_returns'],
                             label='Return')
    _ = trend_return_ax.set_title(f'{ticker} Trend Strategy with Donchian Channel Return')
    _ = trend_return_ax.set_ylabel('Return')
    _ = trend_return_ax.set_xlabel('Date')
    _ = trend_return_ax.legend(loc='upper left')
    _ = trend_return_ax.grid()

    # _ = trend_cum_return_ax.plot(df_trend_mavg_donchian.index, df_trend_mavg_donchian[f'{ticker}_trend_strategy_returns_{fast_mavg}_{mavg_stepsize}_{slow_mavg}_cum'], label='Cum. Return')
    _ = trend_cum_return_ax.plot(df.index, df['strategy_cumulative_return'], label='Cum. Return')
    _ = trend_cum_return_ax.set_title(f'{ticker} Trend Strategy with Donchian Channel Cumulative Return')
    _ = trend_cum_return_ax.set_ylabel('Cum. Return')
    _ = trend_cum_return_ax.set_xlabel('Date')
    _ = trend_cum_return_ax.legend(loc='upper left')
    _ = trend_cum_return_ax.grid()


    plt.tight_layout()
    
    return

## Save Crypto Data from Coinbase

In [13]:
ticker_list_save = ['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD','XRP-USD','AAVE-USD','MATIC-USD']

In [ ]:
for ticker in ticker_list_save:
    print(ticker)
    df = cn.save_historical_crypto_prices_from_coinbase(ticker=ticker, user_start_date=False, start_date=None, 
                                                        end_date=pd.Timestamp('2025-07-31').date(), save_to_file=True)

In [ ]:
df.tail()

## Trend Following Signal

In [15]:
from collections import OrderedDict

def print_strategy_params():
    """
    Pretty-print the strategy’s configuration values, with a blank line
    separating each logical section.
    """

    # ---- Define sections (title is just for dev readability) --------------
    sections = [
        ("Dates & universe", OrderedDict([
            ("start_date",      start_date),
            ("end_date",        end_date),
            ("warm_up_days",    WARMUP_DAYS),
            ("ticker_list",     ticker_list),
        ])),

        ("Moving-average / trend", OrderedDict([
            ("fast_mavg",                  fast_mavg),
            ("slow_mavg",                  slow_mavg),
            ("mavg_stepsize",              mavg_stepsize),
            ("mavg_z_score_window",        mavg_z_score_window),
            ("moving_avg_type",            moving_avg_type),
            ("ma_crossover_signal_weight", ma_crossover_signal_weight),
        ])),

        ("Donchian channel", OrderedDict([
            ("entry_rolling_donchian_window", entry_rolling_donchian_window),
            ("exit_rolling_donchian_window", exit_rolling_donchian_window),
            ("use_donchian_exit_gate", use_donchian_exit_gate),
            ("donchian_signal_weight",  donchian_signal_weight),
        ])),

        ("Volatility & risk", OrderedDict([
            ("volatility_window",            volatility_window),
            ("annualized_target_volatility", annualized_target_volatility),
            ("rolling_cov_window",           rolling_cov_window),
            ("rolling_atr_window",           rolling_atr_window),
            ("atr_multiplier",               atr_multiplier),
            ("log_std_window",               log_std_window),
            ("coef_of_variation_window",     coef_of_variation_window),
            ("vol_of_vol_z_score_window",    vol_of_vol_z_score_window),
            ("vol_of_vol_p_min",             vol_of_vol_p_min)
        ])),

        ("Signal gating / quality", OrderedDict([
            ("lower_r_sqr_limit",   lower_r_sqr_limit),
            ("upper_r_sqr_limit",   upper_r_sqr_limit),
            ("r2_window",           r2_window),
            ("rolling_sharpe_window", rolling_sharpe_window),
            ("use_activation", use_activation),
            ("tanh_activation_constant_dict", tanh_activation_constant_dict),
        ])),

        ("Trading toggles & thresholds", OrderedDict([
            ("long_only",                 long_only),
            ("use_coinbase_data",         use_coinbase_data),
            ("use_saved_files",           use_saved_files),
            ("saved_file_end_date",       saved_file_end_date),
            ("use_specific_start_date",   use_specific_start_date),
            ("signal_start_date",         signal_start_date),
            ("price_or_returns_calc",     price_or_returns_calc),
            ("notional_threshold_pct",    notional_threshold_pct),
            ("cooldown_counter_threshold", cooldown_counter_threshold),
        ])),

        ("Capital & execution", OrderedDict([
            ("initial_capital",        initial_capital),
            ("cash_buffer_percentage", cash_buffer_percentage),
            ("transaction_cost_est",   transaction_cost_est),
            ("passive_trade_rate",     passive_trade_rate),
            ("annual_trading_days",    annual_trading_days),
        ])),
    ]

    # ---- Compute width for neat alignment ---------------------------------
    longest_key = max(len(k) for _, sec in sections for k in sec)

    print("\nStrategy Parameters\n" + "-" * (longest_key + 30))
    for _, sec in sections:
        for k, v in sec.items():
            print(f"{k:<{longest_key}} : {v}")
        print()  # blank line between sections
    print("-" * (longest_key + 30) + "\n")

# ---------------------------------------------------------------------------
# Example usage (uncomment after your own parameter definitions are in scope)
# ---------------------------------------------------------------------------
# if __name__ == "__main__":
#     print_strategy_params()

In [17]:
def plot_signal_performance(df_1, df_2, ticker):

    fig = plt.figure(figsize=(20,12))
    layout = (2,2)
    signal_ax = plt.subplot2grid(layout, (0,0))
    price_ax = signal_ax.twinx()
    equity_curve_ax = plt.subplot2grid(layout, (0,1))
    sharpe_ax = plt.subplot2grid(layout, (1,0))
    portfolio_value_ax = plt.subplot2grid(layout, (1,1))

    _ = signal_ax.plot(df_1.index, df_1[f'{ticker}_final_signal'], label='Orig Signal', alpha=0.9)
    _ = signal_ax.plot(df_2.index, df_2[f'{ticker}_final_signal'], label='New Signal', alpha=0.9)
    _ = price_ax.plot(df_1.index, df_2[f'{ticker}_open'], label='Price', alpha=0.7, linestyle='--', color='magenta')
    _ = signal_ax.set_title(f'Orignal Signal vs New Signal')
    _ = signal_ax.set_ylabel('Signal')
    _ = signal_ax.set_xlabel('Date')
    _ = signal_ax.legend(loc='upper left')
    _ = signal_ax.grid()

    _ = equity_curve_ax.plot(df_1.index, df_1[f'equity_curve'], label='Orig Signal', alpha=0.9)
    _ = equity_curve_ax.plot(df_2.index, df_2[f'equity_curve'], label='New Signal', alpha=0.9)
    _ = equity_curve_ax.set_title(f'Equity Curve')
    _ = equity_curve_ax.set_ylabel('Equity Curve')
    _ = equity_curve_ax.set_xlabel('Date')
    _ = equity_curve_ax.legend(loc='upper left')
    _ = equity_curve_ax.grid()

    _ = sharpe_ax.plot(df_1.index, df_1[f'portfolio_rolling_sharpe_50'], label='Orig Signal', alpha=0.9)
    _ = sharpe_ax.plot(df_2.index, df_2[f'portfolio_rolling_sharpe_50'], label='New Signal', alpha=0.9)
    _ = sharpe_ax.set_title(f'Rolling Sharpe')
    _ = sharpe_ax.set_ylabel(f'Rolling Sharpe')
    _ = sharpe_ax.set_xlabel('Date')
    _ = sharpe_ax.legend(loc='upper left')
    _ = sharpe_ax.grid()

    _ = portfolio_value_ax.plot(df_1.index, df_1[f'total_portfolio_value'], label='Orig Signal', alpha=0.9)
    _ = portfolio_value_ax.plot(df_2.index, df_2[f'total_portfolio_value'], label='New Signal', alpha=0.9)
    _ = portfolio_value_ax.set_title(f'Total Portfolio Value')
    _ = portfolio_value_ax.set_ylabel('Portfolio Value')
    _ = portfolio_value_ax.set_xlabel('Date')
    _ = portfolio_value_ax.legend(loc='upper left')
    _ = portfolio_value_ax.grid()

    plt.tight_layout()

    return

In [19]:
from scipy.stats import linregress

def calc_ribbon_slope(row, ticker, fast_mavg, slow_mavg, mavg_stepsize):
    x = np.linspace(slow_mavg, fast_mavg, mavg_stepsize)
    y = row.values
    slope, _, _, _, _ = linregress(x, y)
    return slope

In [21]:
def pct_rank(x, window=250):
    return x.rank(pct=True)

In [23]:
def calculate_average_directional_index(start_date, end_date, ticker, adx_period):
    
    ## Convert number of bars to days. ## alpha = 2/(span + 1) for the Exponentially Weighted Average
    ## If alpha = 1/n, span = 2*n - 1
    adx_atr_window = 2*adx_period - 1

    ## Pull Market Data
    if use_coinbase_data:
        # df = cn.get_coinbase_ohlc_data(ticker=ticker)
        df = cn.save_historical_crypto_prices_from_coinbase(ticker=ticker, end_date=end_date, save_to_file=False)
        df = df[(df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)]
        df.columns = [f'{ticker}_{x}' for x in df.columns]
    else:
        df = tf.load_financial_data(start_date, end_date, ticker, print_status=False)  # .shift(1)
        df.columns = [f'{ticker}_open', f'{ticker}_high', f'{ticker}_low', f'{ticker}_close', f'{ticker}_adjclose',
                      f'{ticker}_volume']
    
    ## Calculate Directional Move
    df[f'{ticker}_up_move'] = df[f'{ticker}_high'].diff()
    df[f'{ticker}_down_move'] = -df[f'{ticker}_low'].diff()
    
    plus_dir_move_cond = (df[f'{ticker}_up_move'] > df[f'{ticker}_down_move']) & (df[f'{ticker}_up_move'] > 0)
    minus_dir_move_cond = (df[f'{ticker}_down_move'] > df[f'{ticker}_up_move']) & (df[f'{ticker}_down_move'] > 0)
    df[f'{ticker}_plus_dir_move'] = np.where(plus_dir_move_cond, df[f'{ticker}_up_move'], 0)
    df[f'{ticker}_minus_dir_move'] = np.where(minus_dir_move_cond, df[f'{ticker}_down_move'], 0)
    
    ## Calculate the True Range (TR) and Average True Range (ATR)
    df[f'{ticker}_high-low'] = df[f'{ticker}_high'] - df[f'{ticker}_low']
    df[f'{ticker}_high-close'] = np.abs(df[f'{ticker}_high'] - df[f'{ticker}_close'].shift(1))
    df[f'{ticker}_low-close'] = np.abs(df[f'{ticker}_low'] - df[f'{ticker}_close'].shift(1))
    df[f'{ticker}_true_range_price'] = df[[f'{ticker}_high-low',f'{ticker}_high-close', f'{ticker}_low-close']].max(axis=1)
    df[f'{ticker}_{adx_atr_window}_avg_true_range'] = df[f'{ticker}_true_range_price'].ewm(span=adx_atr_window, adjust=False).mean()
    
    ## Calculate the exponentially weighted directional moves
    df[f'{ticker}_plus_dir_move_exp'] = df[f'{ticker}_plus_dir_move'].ewm(span=adx_atr_window, adjust=False).mean()
    df[f'{ticker}_minus_dir_move_exp'] = df[f'{ticker}_minus_dir_move'].ewm(span=adx_atr_window, adjust=False).mean()
    
    ## Calculate the directional indicator
    df[f'{ticker}_plus_dir_ind'] = 100 * (df[f'{ticker}_plus_dir_move_exp'] / df[f'{ticker}_{adx_atr_window}_avg_true_range'])
    df[f'{ticker}_minus_dir_ind'] = 100 * (df[f'{ticker}_minus_dir_move_exp'] / df[f'{ticker}_{adx_atr_window}_avg_true_range'])
    df[f'{ticker}_dir_ind'] = 100 * np.abs((df[f'{ticker}_plus_dir_ind'] - df[f'{ticker}_minus_dir_ind'])) / (df[f'{ticker}_plus_dir_ind'] + df[f'{ticker}_minus_dir_ind'])
    df[f'{ticker}_avg_dir_ind'] = df[f'{ticker}_dir_ind'].ewm(span=adx_atr_window, adjust=False).mean()
    
    ## Shift by a day to avoid look-ahead bias
    df[f'{ticker}_avg_dir_ind'] = df[f'{ticker}_avg_dir_ind'].shift(1)

    return df[[f'{ticker}_avg_dir_ind']]

In [458]:
import scipy

def create_trend_strategy_log_space(df, ticker, mavg_start, mavg_end, mavg_stepsize, mavg_z_score_window=252):
    
    # ---- constants ----
    # windows = 2**np.arange(np.log2(mavg_start), np.log2(mavg_end)+0.01, log_step).round().astype(int)   # e.g. 10,14,20,28,40,56
    windows = np.geomspace(mavg_start, mavg_end, mavg_stepsize).round().astype(int)
    windows = np.unique(windows)
    x       = np.log(windows[::-1])
    xm      = x - x.mean()
    varx    = (xm**2).sum()
    
    # ---- compute MAs (vectorised) ----
    df[f'{ticker}_close_log'] = np.log(df[f'{ticker}_close'])
    for w in windows:
        df[f'{ticker}_{w}_ema'] = df[f'{ticker}_close_log'].ewm(span=w, adjust=False).mean()
    
    mavg_mat = df[[f'{ticker}_{w}_ema' for w in windows]].to_numpy()
    
    # ---- slope (vectorised) ----
    slope = mavg_mat.dot(xm) / varx                        # ndarray (T,)
    slope = pd.Series(slope, index=df.index).shift(1)      # lag to avoid look-ahead
    
    # ---- z-score & rank ----
    # z = ((slope - slope.rolling(mavg_z_score_window, min_periods=mavg_z_score_window//5).mean()) /
    #      slope.rolling(mavg_z_score_window, min_periods=mavg_z_score_window//5).std())
    # Removing the Min Period, the Z-Score will be calculated when the mavg_z_score_window history is available
    z = ((slope - slope.rolling(mavg_z_score_window, min_periods=mavg_z_score_window).mean()) /
         slope.rolling(mavg_z_score_window, min_periods=mavg_z_score_window).std())
    

    # Optional Tail Cap
    z = z.clip(-4, 4)

    # Calculate the Percentile Rank based on CDF
    rank = scipy.stats.norm.cdf(z) - 0.5              # centred 0↔±0.5
    
    trend_continuous_signal_col = f'{ticker}_mavg_ribbon_slope'
    trend_continuous_signal_rank_col = f'{ticker}_mavg_ribbon_rank'
    df[trend_continuous_signal_col] = slope
    df[trend_continuous_signal_rank_col] = rank

    return df

In [711]:
def calculate_donchian_channel_dual_window(start_date, end_date, ticker, price_or_returns_calc='price', entry_rolling_donchian_window=20, exit_rolling_donchian_window=20, 
                                           use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30'):
    if use_coinbase_data:
        if use_saved_files:
            date_list = cn.coinbase_start_date_by_ticker_dict
            file_end_date = pd.Timestamp(saved_file_end_date).date()
            filename = f"{ticker}-pickle-{pd.Timestamp(date_list[ticker]).strftime('%Y-%m-%d')}-{file_end_date.strftime('%Y-%m-%d')}"
            output_file = f'coinbase_historical_price_folder/{filename}'
            df = pd.read_pickle(output_file)
            date_cond = (df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)
            df = df[date_cond]
        else:
            # df = cn.get_coinbase_ohlc_data(ticker=ticker)
            df = cn.save_historical_crypto_prices_from_coinbase(ticker=ticker, user_start_date=True, start_date=start_date,
                                                                end_date=end_date, save_to_file=False)
            df = df[(df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)]
    else:
        df = load_financial_data(start_date, end_date, ticker, print_status=False)  # .shift(1)
        df.columns = ['open', 'high', 'low', 'close', 'adjclose', 'volume']

    if price_or_returns_calc == 'price':
        ## Entry Channel
        # Rolling maximum of returns (upper channel)
        df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_upper_band_{price_or_returns_calc}'] = (
            df[f'high'].rolling(window=entry_rolling_donchian_window).max())

        # Rolling minimum of returns (lower channel)
        df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_lower_band_{price_or_returns_calc}'] = (
            df[f'low'].rolling(window=entry_rolling_donchian_window).min())

        ## Exit Channel
        # Rolling maximum of returns (upper channel)
        df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_upper_band_{price_or_returns_calc}'] = (
            df[f'high'].rolling(window=exit_rolling_donchian_window).max())

        # Rolling minimum of returns (lower channel)
        df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_lower_band_{price_or_returns_calc}'] = (
            df[f'low'].rolling(window=exit_rolling_donchian_window).min())

    elif price_or_returns_calc == 'returns':
        # Calculate Percent Returns
        df[f'{ticker}_pct_returns'] = df[f'close'].pct_change()

        ## Entry Channel
        # Rolling maximum of returns (upper channel)
        df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_upper_band_{price_or_returns_calc}'] = df[
            f'{ticker}_pct_returns'].rolling(window=entry_rolling_donchian_window).max()

        # Rolling minimum of returns (lower channel)
        df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_lower_band_{price_or_returns_calc}'] = df[
            f'{ticker}_pct_returns'].rolling(window=entry_rolling_donchian_window).min()

        ## Exit Channel
        # Rolling maximum of returns (upper channel)
        df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_upper_band_{price_or_returns_calc}'] = df[
            f'{ticker}_pct_returns'].rolling(window=exit_rolling_donchian_window).max()

        # Rolling minimum of returns (lower channel)
        df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_lower_band_{price_or_returns_calc}'] = df[
            f'{ticker}_pct_returns'].rolling(window=exit_rolling_donchian_window).min()

    # Middle of the channel (optional, could be just average of upper and lower)
    # Entry Middle Band
    df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_middle_band_{price_or_returns_calc}'] = (
        (df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_upper_band_{price_or_returns_calc}'] +
         df[f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_lower_band_{price_or_returns_calc}']) / 2)

    # Exit Middle Band
    df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_middle_band_{price_or_returns_calc}'] = (
        (df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_upper_band_{price_or_returns_calc}'] +
         df[f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_lower_band_{price_or_returns_calc}']) / 2)

    # Shift only the Keltner channel metrics to avoid look-ahead bias
    shift_columns = [
        f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_middle_band_{price_or_returns_calc}',
        f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_upper_band_{price_or_returns_calc}',
        f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_lower_band_{price_or_returns_calc}',
        f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_middle_band_{price_or_returns_calc}',
        f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_upper_band_{price_or_returns_calc}',
        f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_lower_band_{price_or_returns_calc}'
    ]
    df[shift_columns] = df[shift_columns].shift(1)

    return df

In [29]:
## Original Signal
def generate_trend_signal_with_donchian_channel_continuous(start_date, end_date, ticker, fast_mavg, slow_mavg, mavg_stepsize, mavg_z_score_window, entry_rolling_donchian_window, 
                                                           exit_rolling_donchian_window, use_donchian_exit_gate, donchian_signal_weight, ma_crossover_signal_weight,
                                                           use_activation=True, tanh_activation_constant_dict=None, 
                                                           moving_avg_type='exponential', price_or_returns_calc='price',
                                                           long_only=False, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30'):

    # Pull Close Prices from Coinbase
    date_list = cn.coinbase_start_date_by_ticker_dict
    if use_saved_files:
        file_end_date = pd.Timestamp(saved_file_end_date).date()
        filename = f"{ticker}-pickle-{pd.Timestamp(date_list[ticker]).strftime('%Y-%m-%d')}-{file_end_date.strftime('%Y-%m-%d')}"
        output_file = f'coinbase_historical_price_folder/{filename}'
        df = pd.read_pickle(output_file)
        df = (df[['close','open']].rename(columns={'close': f'{ticker}_close', 'open': f'{ticker}_open'}))
        date_cond = (df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)
        df = df[date_cond]
    else:
        df = cn.save_historical_crypto_prices_from_coinbase(ticker=ticker, user_start_date=True, start_date=start_date,
                                                            end_date=end_date, save_to_file=False)
        df = (df[['close','open']].rename(columns={'close': f'{ticker}_close', 'open': f'{ticker}_open'}))
        date_cond = (df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)
        df = df[date_cond]
    
    # Create Column Names
    donchian_binary_signal_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_binary_signal'
    donchian_continuous_signal_col = f'{ticker}_donchian_continuous_signal'
    donchian_continuous_signal_rank_col = f'{ticker}_donchian_continuous_signal_rank'
    trend_binary_signal_col = f'{ticker}_trend_signal'
    trend_continuous_signal_col = f'{ticker}_mavg_ribbon_slope'
    trend_continuous_signal_rank_col = f'{ticker}_mavg_ribbon_rank'
    final_binary_signal_col = f'{ticker}_final_binary_signal'
    final_weighted_additive_signal_col = f'{ticker}_final_weighted_additive_signal'
    final_signal_col = f'{ticker}_final_signal'

    ## Generate Trend Signal in Log Space
    df_trend = create_trend_strategy_log_space(df, ticker, mavg_start=fast_mavg, mavg_end=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window)
    
    ## Generate Donchian Channels
    # Donchian Buy signal: Price crosses above upper band
    # Donchian Sell signal: Price crosses below lower band
    df_donchian = calculate_donchian_channel_dual_window(start_date=start_date, end_date=end_date, ticker=ticker, price_or_returns_calc=price_or_returns_calc,
                                                         entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window,
                                                         use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)

    t_1_close_col = f't_1_close'
    df_donchian[t_1_close_col] = df_donchian[f'close'].shift(1)
    donchian_entry_upper_band_col = f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_upper_band_{price_or_returns_calc}'
    donchian_entry_lower_band_col = f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_lower_band_{price_or_returns_calc}'
    donchian_entry_middle_band_col = f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_middle_band_{price_or_returns_calc}'
    donchian_exit_upper_band_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_upper_band_{price_or_returns_calc}'
    donchian_exit_lower_band_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_lower_band_{price_or_returns_calc}'
    donchian_exit_middle_band_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_middle_band_{price_or_returns_calc}'
    shift_cols = [donchian_entry_upper_band_col, donchian_entry_lower_band_col, donchian_entry_middle_band_col,
                  donchian_exit_upper_band_col, donchian_exit_lower_band_col, donchian_exit_middle_band_col]
    for col in shift_cols:
        df_donchian[f'{col}_t_2'] = df_donchian[col].shift(1)

    # Donchian Continuous Signal
    df_donchian[donchian_continuous_signal_col] = ((df_donchian[t_1_close_col] - df_donchian[f'{donchian_entry_middle_band_col}_t_2']) /
                                                   (df_donchian[f'{donchian_entry_upper_band_col}_t_2'] - df_donchian[f'{donchian_entry_lower_band_col}_t_2']))

    ## Calculate Donchian Channel Rank
    ## Adjust the percentage ranks by 0.5 as without, the ranks go from 0 to 1. Recentering the function by giving it a steeper 
    ## slope near the origin takes into account even little information
    df_donchian[donchian_continuous_signal_rank_col] = pct_rank(df_donchian[donchian_continuous_signal_col]) - 0.5

    # Donchian Binary Signal
    gate_long_condition  = df_donchian[t_1_close_col] >= df_donchian[f'{donchian_exit_lower_band_col}_t_2']
    gate_short_condition = df_donchian[t_1_close_col] <= df_donchian[f'{donchian_exit_upper_band_col}_t_2']
    # sign of *entry* score decides direction
    entry_sign = np.sign(df_donchian[donchian_continuous_signal_col])
    # treat exact zero as "flat but allowed" (gate=1) so ranking not wiped out
    entry_sign = np.where(entry_sign == 0, 1, entry_sign)  # default to long-side keep
    df_donchian[donchian_binary_signal_col] = np.where(
        entry_sign > 0, gate_long_condition, gate_short_condition).astype(float)
    
    # Merging the Trend and Donchian Dataframes
    donchian_cols = [f'{donchian_entry_upper_band_col}_t_2', f'{donchian_entry_lower_band_col}_t_2', f'{donchian_entry_middle_band_col}_t_2',
                     f'{donchian_exit_upper_band_col}_t_2', f'{donchian_exit_lower_band_col}_t_2', f'{donchian_exit_middle_band_col}_t_2',
                     donchian_binary_signal_col, donchian_continuous_signal_col, donchian_continuous_signal_rank_col]
    df_trend = pd.merge(df_trend, df_donchian[donchian_cols], left_index=True, right_index=True, how='left')

    ## Trend and Donchian Channel Signal
    # Calculate the exponential weighted average of the ranked signals to remove short-term flip flops (whiplash)
    df_trend[[trend_continuous_signal_rank_col, donchian_continuous_signal_rank_col]] = (
        df_trend[[trend_continuous_signal_rank_col, donchian_continuous_signal_rank_col]].ewm(span=3, adjust=False).mean())

    # Weighted Sum of Rank Columns
    df_trend[final_weighted_additive_signal_col] = (ma_crossover_signal_weight * df_trend[trend_continuous_signal_rank_col] +
                                                    donchian_signal_weight * df_trend[donchian_continuous_signal_rank_col])

    # Activation Scaled Signal
    if use_activation:
        final_signal_unscaled_95th_percentile = np.abs(df_trend[final_weighted_additive_signal_col]).quantile(0.95)
        if tanh_activation_constant_dict:
            k = tanh_activation_constant_dict[ticker]
            df_trend[f'{ticker}_activation'] = np.tanh(df_trend[final_weighted_additive_signal_col] * k)
        else:
            if (final_signal_unscaled_95th_percentile == 0):#| (final_signal_unscaled_95th_percentile.isnan()):
                k = 1.0
            else:
                k = np.arctanh(0.9) / final_signal_unscaled_95th_percentile
            df_trend[f'{ticker}_activation'] = np.tanh(df_trend[final_weighted_additive_signal_col] * k)
    else:
        df_trend[f'{ticker}_activation'] = df_trend[final_weighted_additive_signal_col]

    # Apply Binary Gate
    if use_donchian_exit_gate:
        df_trend[f'{ticker}_activation'] = df_trend[f'{ticker}_activation'] * df_trend[donchian_binary_signal_col]

    ## Long-Only Filter
    df_trend[final_signal_col] = np.where(long_only, np.maximum(0, df_trend[f'{ticker}_activation']), df_trend[f'{ticker}_activation'])

    return df_trend

def get_trend_donchian_signal_for_portfolio(start_date, end_date, ticker_list, fast_mavg, slow_mavg, mavg_stepsize, mavg_z_score_window, entry_rolling_donchian_window, 
                                            exit_rolling_donchian_window, use_donchian_exit_gate, donchian_signal_weight, ma_crossover_signal_weight, 
                                            use_activation=True, tanh_activation_constant_dict=None, 
                                            long_only=False, price_or_returns_calc='price',
                                            use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30'):

    ## Generate trend signal for all tickers
    trend_list = []
    date_list = cn.coinbase_start_date_by_ticker_dict
    
    for ticker in ticker_list:
        # Create Column Names
        donchian_continuous_signal_col = f'{ticker}_donchian_continuous_signal'
        donchian_continuous_signal_rank_col = f'{ticker}_donchian_continuous_signal_rank'
        trend_continuous_signal_col = f'{ticker}_mavg_ribbon_slope'
        trend_continuous_signal_rank_col = f'{ticker}_mavg_ribbon_rank'
        final_signal_col = f'{ticker}_final_signal'
        close_price_col = f'{ticker}_close'
        open_price_col = f'{ticker}_open'
        final_weighted_additive_signal_col = f'{ticker}_final_weighted_additive_signal'
        # lower_donchian_col = f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}_t_2'
        # upper_donchian_col = f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}_t_2'
        
        if pd.to_datetime(date_list[ticker]).date() > start_date:
            df_trend = generate_trend_signal_with_donchian_channel_continuous(
                start_date=pd.to_datetime(date_list[ticker]).date(), end_date=end_date, ticker=ticker,
                fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window,
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window,
                use_donchian_exit_gate=use_donchian_exit_gate, donchian_signal_weight=donchian_signal_weight, 
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, price_or_returns_calc=price_or_returns_calc, long_only=long_only,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)
        else:
            df_trend = generate_trend_signal_with_donchian_channel_continuous(
                start_date=start_date, end_date=end_date, ticker=ticker, fast_mavg=fast_mavg, slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize,
                mavg_z_score_window=mavg_z_score_window,
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window,
                use_donchian_exit_gate=use_donchian_exit_gate, donchian_signal_weight=donchian_signal_weight, 
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, price_or_returns_calc=price_or_returns_calc, long_only=long_only,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)
            
        trend_cols = [close_price_col, open_price_col, trend_continuous_signal_col, trend_continuous_signal_rank_col, final_weighted_additive_signal_col, final_signal_col]
        # trend_cols = [close_price_col, open_price_col, lower_donchian_col, upper_donchian_col, donchian_continuous_signal_col, donchian_continuous_signal_rank_col,
        #               trend_continuous_signal_col, trend_continuous_signal_rank_col, final_weighted_additive_signal_col, final_signal_col]
        df_trend = df_trend[trend_cols]
        trend_list.append(df_trend)

    df_trend = pd.concat(trend_list, axis=1)

    return df_trend

In [31]:
def apply_target_volatility_position_sizing_continuous_strategy(start_date, end_date, ticker_list, fast_mavg, slow_mavg, mavg_stepsize, mavg_z_score_window, ma_crossover_signal_weight,
                                                                donchian_signal_weight, entry_rolling_donchian_window, exit_rolling_donchian_window, use_donchian_exit_gate, 
                                                                use_activation=True, tanh_activation_constant_dict=None, long_only=False,
                                                                initial_capital=15000, rolling_cov_window=20, volatility_window=20,
                                                                rolling_atr_window=20, atr_multiplier=0.5,
                                                                transaction_cost_est=0.001, passive_trade_rate=0.05,
                                                                use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30', 
                                                                rolling_sharpe_window=50, cash_buffer_percentage=0.10, annualized_target_volatility=0.20,
                                                                annual_trading_days=365, use_specific_start_date=False,
                                                                signal_start_date=None):

    ## Check if data is available for all the tickers
    date_list = cn.coinbase_start_date_by_ticker_dict
    ticker_list = [ticker for ticker in ticker_list if pd.Timestamp(date_list[ticker]).date() < end_date]
    
    print('Generating Moving Average Ribbon Signal!!')
    ## Generate Trend Signal for all tickers
    df_trend = get_trend_donchian_signal_for_portfolio(start_date=start_date, end_date=end_date, ticker_list=ticker_list, fast_mavg=fast_mavg,
                                                       slow_mavg=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                                                       entry_rolling_donchian_window=entry_rolling_donchian_window, 
                                                       exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate,
                                                       donchian_signal_weight=donchian_signal_weight, ma_crossover_signal_weight=ma_crossover_signal_weight, 
                                                       use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                                                       long_only=long_only, use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)

    print('Generating Volatility Adjusted Trend Signal!!')
    ## Get Volatility Adjusted Trend Signal
    df_signal = size_cont.get_volatility_adjusted_trend_signal_continuous(df_trend, ticker_list, volatility_window, annual_trading_days)

    print('Getting Average True Range for Stop Loss Calculation!!')
    ## Get Average True Range for Stop Loss Calculation
    df_atr = size_cont.get_average_true_range_portfolio(start_date=start_date, end_date=end_date, ticker_list=ticker_list, rolling_atr_window=rolling_atr_window,
                                                        price_or_returns_calc='price', use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files,
                                                        saved_file_end_date=saved_file_end_date)
    df_signal = pd.merge(df_signal, df_atr, left_index=True, right_index=True, how='left')

    print('Calculating Volatility Targeted Position Size and Cash Management!!')
    ## Get Target Volatility Position Sizing and Run Cash Management
    df = size_cont.get_target_volatility_daily_portfolio_positions(df_signal, ticker_list=ticker_list, initial_capital=initial_capital, rolling_cov_window=rolling_cov_window,
                                                                   rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier, cash_buffer_percentage=cash_buffer_percentage,
                                                                   annualized_target_volatility=annualized_target_volatility, transaction_cost_est=transaction_cost_est,
                                                                   passive_trade_rate=passive_trade_rate, notional_threshold_pct=notional_threshold_pct,
                                                                   cooldown_counter_threshold=cooldown_counter_threshold, annual_trading_days=annual_trading_days,
                                                                   use_specific_start_date=use_specific_start_date, signal_start_date=signal_start_date)

    print('Calculating Portfolio Performance!!')
    ## Calculate Portfolio Performance
    df = size_bin.calculate_portfolio_returns(df, rolling_sharpe_window)

    return df

In [33]:
def calculate_asset_level_returns(df, end_date, ticker_list):
    ## Check if data is available for all the tickers
    date_list = cn.coinbase_start_date_by_ticker_dict
    ticker_list = [ticker for ticker in ticker_list if pd.Timestamp(date_list[ticker]).date() < end_date]
    
    for ticker in ticker_list:
        df[f'{ticker}_daily_pnl'] = (df[f'{ticker}_actual_position_size'] * df[f'{ticker}_open'].diff().shift(-1))
        df[f'{ticker}_daily_pct_returns'] = (df[f'{ticker}_daily_pnl'] / df[f'total_portfolio_value'].shift(1)).fillna(0)
        df[f'{ticker}_position_count'] = np.where((df[f'{ticker}_actual_position_notional'] != 0), 1, 0) ## This is not entirely accurate
    return df

## Moving Average Ribbon Optimization

In [35]:
start_date = pd.to_datetime('2021-06-01').date()
end_date = pd.to_datetime('2023-12-31').date()
start_date_os = pd.to_datetime('2024-01-01').date()
end_date_os = pd.to_datetime('2025-06-30').date()
WARMUP_DAYS = 285
ticker_list = ['BTC-USD','ETH-USD','SOL-USD','ADA-USD','AVAX-USD']#,'XRP-USD','AAVE-USD']
fast_mavg = 16
slow_mavg = 224
mavg_stepsize = 8
mavg_z_score_window = 126
entry_rolling_donchian_window = 28
exit_rolling_donchian_window = 28
use_donchian_exit_gate = False
long_only = True
use_coinbase_data = True
use_saved_files = True
saved_file_end_date = '2025-06-30'
volatility_window = 20
annual_trading_days = 365
rolling_cov_window = 20
annualized_target_volatility = 0.70
rolling_atr_window = 20
atr_multiplier = 2.0
use_specific_start_date = False
signal_start_date = None
initial_capital = 15000
cash_buffer_percentage = 0.10
transaction_cost_est = 0.001#0.0025
passive_trade_rate = 0.05
notional_threshold_pct = 0.05
cooldown_counter_threshold = 3
rolling_sharpe_window = 50
price_or_returns_calc = 'price'
moving_avg_type = 'exponential'
use_coinbase_data = True

ma_crossover_signal_weight = 0.6
donchian_signal_weight = 0.4
use_activation = False
tanh_activation_constant_dict = None
lower_r_sqr_limit = 0.30
upper_r_sqr_limit = 0.90
r2_window = 80
log_std_window = 14
coef_of_variation_window = 30
vol_of_vol_z_score_window = 252
vol_of_vol_p_min = 0.6
use_specific_start_date = True
signal_start_date = pd.Timestamp('2021-06-01').date()

In [37]:
print_strategy_params()


Strategy Parameters
-----------------------------------------------------------
start_date                    : 2021-06-01
end_date                      : 2023-12-31
warm_up_days                  : 285
ticker_list                   : ['BTC-USD', 'ETH-USD', 'SOL-USD', 'ADA-USD', 'AVAX-USD']

fast_mavg                     : 16
slow_mavg                     : 224
mavg_stepsize                 : 8
mavg_z_score_window           : 126
moving_avg_type               : exponential
ma_crossover_signal_weight    : 0.6

entry_rolling_donchian_window : 28
exit_rolling_donchian_window  : 28
use_donchian_exit_gate        : False
donchian_signal_weight        : 0.4

volatility_window             : 20
annualized_target_volatility  : 0.7
rolling_cov_window            : 20
rolling_atr_window            : 20
atr_multiplier                : 2.0
log_std_window                : 14
coef_of_variation_window      : 30
vol_of_vol_z_score_window     : 252
vol_of_vol_p_min              : 0.6

lower_r_sqr_limit   

## Run Walk Forward Analysis for Moving Average Crossover Signal

In [215]:
import itertools

def generate_moving_avg_ribbon_params():
    parameter_grid = {
        "fast_window": [8, 10, 12, 14, 16, 18, 20],
        "slow_fast_ratio":[4, 6, 8, 10, 12, 14, 16],
        "stepsize":[8],
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [223]:
def run_walk_forward_moving_avg_ribbon(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'fast_mavg', 'slow_mavg', 'mavg_stepsize', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        # print(f'In Sample Start: {start_date_is}, In Sample End: {end_date_is}, Out of Sample Start: {start_date_os}, Out of Sample End: {end_date_os}')
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        for params in generate_moving_avg_ribbon_params():
            print(params)
            fast_mavg = params['fast_window']
            slow_mavg = params['slow_fast_ratio'] * fast_mavg
            mavg_stepsize = params['stepsize']
            print(fast_mavg, slow_mavg, mavg_stepsize)
            
            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = apply_target_volatility_position_sizing_continuous_strategy(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
                use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
                annualized_target_volatility=annualized_target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= start_date_is]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = calculate_asset_level_returns(df_is, end_date, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'fast_mavg': fast_mavg,
                'slow_mavg': slow_mavg,
                'mavg_stepsize': mavg_stepsize
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)

            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample and out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        ## Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        best_in_sample_fast_mavg = df_performance[in_sample_cond & date_cond].sort_values('annualized_sharpe_ratio', ascending=False)['fast_mavg'].iloc[0]
        best_in_sample_slow_mavg = df_performance[in_sample_cond & date_cond].sort_values('annualized_sharpe_ratio', ascending=False)['slow_mavg'].iloc[0]
        print(f'Best In Sample Fast Mavg: {best_in_sample_fast_mavg}')
        print(f'Best In Sample Slow Mavg: {best_in_sample_slow_mavg}')

        ## Out of Sample Dataframe
        df_os = apply_target_volatility_position_sizing_continuous_strategy(
            start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=best_in_sample_fast_mavg, slow_mavg=best_in_sample_slow_mavg,
            mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
            use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
            initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
            rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
            transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
            rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
            annualized_target_volatility=annualized_target_volatility,
            annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_os)

        df_os = df_os[df_os.index >= start_date_os]
        print('Calculating Out of Sample Asset Returns!!')
        df_os = calculate_asset_level_returns(df_os, end_date, ticker_list)

        ## Out of Sample Performance Metrics
        print('Pulling Out of Sample Performance Metrics!!')
        row_parameters_os = {
            'sampling_category': 'out_sample',
            'start_date': start_date_os,
            'end_date': end_date_os,
            'fast_mavg': best_in_sample_fast_mavg,
            'slow_mavg': best_in_sample_slow_mavg,
            'mavg_stepsize': mavg_stepsize
        }
        portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                           strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                           passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

        print('Getting Out of Sample Asset Performance!!')
        for ticker in ticker_list:
            ## Out of Sample
            ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                 strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                 annual_trading_days=365, include_transaction_costs_and_fees=False)
            ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
            ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
            portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
        
        row_parameters_os.update(portfolio_perf_metrics_os)

        ## Assign in sample and out of sample metrics to performance dataframe
        df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()

    return df_performance

In [225]:
start_date = pd.Timestamp('2022-04-01').date()
end_date = pd.Timestamp('2025-10-01').date()
perf_cols = ['sampling_category', 'start_date', 'end_date', 'fast_mavg', 'slow_mavg', 'mavg_stepsize', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
             'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])

df_performance = pd.DataFrame(columns=perf_cols)

IS_LEN = pd.DateOffset(months=18)
OS_LEN = pd.DateOffset(months=6)
start_date_is = start_date
last_available_date = pd.Timestamp('2025-07-31').date()
WARMUP_DAYS = 323
while True:
    end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
    start_date_os = (end_date_is + pd.Timedelta(days=1))
    end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
    fmt = "%Y-%m-%d"
    
    fields = [
        ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
        ("IS start",          start_date_is),
        ("IS end",            end_date_is),
        ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
        ("OS start",          start_date_os),
        ("OS end",            end_date_os),
    ]
    
    print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
    # print(f'In Sample Start: {start_date_is}, In Sample End: {end_date_is}, Out of Sample Start: {start_date_os}, Out of Sample End: {end_date_os}')
    if end_date_os > end_date - pd.Timedelta(days=1):
        break

    if end_date_os > last_available_date:
        print('end_date_os > last_available_date')
        end_date_os = last_available_date
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
    
    print("Run Dates: ")
    print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
    
    start_date_is = (start_date_is + OS_LEN).date()

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS s

In [227]:
%%time
df_performance_1 = run_walk_forward_moving_avg_ribbon(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'fast_window': 8, 'slow_fast_ratio': 4, 'stepsize': 8}
8 32 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 8, 'slow_fast_ratio': 6, 'stepsize': 8}
8 48 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatili

In [228]:
df_performance_1.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance-2022-04-01-2024-04-01.pickle')

In [229]:
%%time
df_performance_2 = run_walk_forward_moving_avg_ribbon(start_date='2022-10-01', end_date='2024-10-01', ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'fast_window': 8, 'slow_fast_ratio': 4, 'stepsize': 8}
8 32 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 8, 'slow_fast_ratio': 6, 'stepsize': 8}
8 48 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatili

In [230]:
df_performance_2.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance-2022-10-01-2024-10-01.pickle')

In [231]:
%%time
df_performance_3 = run_walk_forward_moving_avg_ribbon(start_date='2023-04-01', end_date='2025-04-01', ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'fast_window': 8, 'slow_fast_ratio': 4, 'stepsize': 8}
8 32 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 8, 'slow_fast_ratio': 6, 'stepsize': 8}
8 48 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatili

In [232]:
df_performance_3.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance-2023-04-01-2025-04-01.pickle')

In [233]:
%%time
df_performance_4 = run_walk_forward_moving_avg_ribbon(start_date='2023-10-01', end_date='2025-10-01', ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'fast_window': 8, 'slow_fast_ratio': 4, 'stepsize': 8}
8 32 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 8, 'slow_fast_ratio': 6, 'stepsize': 8}
8 48 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss 

In [234]:
df_performance_4.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance-2023-10-01-2025-10-01.pickle')

In [ ]:
# %%time
# df_performance_5 = run_walk_forward_moving_avg_ribbon(start_date='2023-06-01', end_date='2025-09-30', ticker_list=ticker_list)

In [ ]:
# df_performance_5.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance-2023-06-01-2025-06-30.pickle')

In [125]:
df_performance_1[df_performance_1.sampling_category == 'in_sample'].sort_values('annualized_sharpe_ratio', ascending=False)

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
7,in_sample,2022-04-01,2023-09-30,10,40,8,0.014463,0.173555,0.027254,0.491599,-0.530680,412 days,0.474453,0.334610,0.738048,1776.0,0.026189,0.101508,-0.037694,-0.000641,0.135580,-0.110789,0.444382,-0.663305,-0.316034,0.645827,0.132792,0.126587,0.121675,0.130595,0.136379,-0.159788,-0.097625,-0.172257,-0.137484,-0.179451
1,in_sample,2022-04-01,2023-09-30,8,48,8,0.011423,0.167908,0.021589,0.500568,-0.529111,412 days,0.470803,0.327534,0.743389,1775.0,0.026825,0.098763,-0.034795,-0.001768,0.136080,-0.106063,0.423809,-0.630091,-0.324901,0.646869,0.132789,0.126873,0.123017,0.130543,0.136964,-0.158153,-0.095050,-0.169347,-0.139879,-0.182405
0,in_sample,2022-04-01,2023-09-30,8,32,8,-0.018879,0.104332,-0.035170,0.494864,-0.536794,412 days,0.479927,0.250149,0.802566,1791.0,0.028079,0.112634,-0.052425,-0.007969,0.128916,-0.100516,0.515590,-0.744288,-0.368285,0.599925,0.130537,0.129488,0.127885,0.132132,0.137120,-0.149533,-0.091384,-0.202678,-0.140883,-0.171438
2,in_sample,2022-04-01,2023-09-30,8,64,8,-0.026550,0.092190,-0.048199,0.502624,-0.550833,412 days,0.485401,0.234174,0.814937,1787.0,0.010173,0.086923,-0.041249,-0.011413,0.145286,-0.233170,0.340135,-0.677357,-0.397273,0.694219,0.132148,0.125746,0.123869,0.131529,0.139876,-0.172352,-0.095041,-0.175259,-0.128582,-0.193293
8,in_sample,2022-04-01,2023-09-30,10,60,8,-0.026311,0.091730,-0.048358,0.500039,-0.544098,412 days,0.487226,0.233915,0.815139,1765.0,0.005416,0.084150,-0.045199,-0.009454,0.147400,-0.270084,0.320724,-0.710064,-0.381579,0.700989,0.131965,0.125018,0.123995,0.131662,0.141485,-0.178708,-0.090916,-0.176181,-0.121209,-0.192471
3,in_sample,2022-04-01,2023-09-30,8,80,8,-0.029237,0.086389,-0.055190,0.503915,-0.529759,412 days,0.478102,0.227182,0.820367,1724.0,0.000165,0.075009,-0.039279,-0.010567,0.157220,-0.314609,0.252227,-0.654723,-0.394478,0.754877,0.130743,0.124955,0.124828,0.130954,0.143023,-0.178324,-0.087625,-0.171172,-0.119683,-0.190568
14,in_sample,2022-04-01,2023-09-30,12,48,8,-0.036977,0.068892,-0.066709,0.493445,-0.554308,412 days,0.474453,0.206017,0.836855,1768.0,0.008540,0.086774,-0.048079,-0.012814,0.139913,-0.245429,0.340037,-0.748891,-0.404234,0.663152,0.132209,0.124810,0.121950,0.132491,0.139142,-0.176604,-0.096364,-0.176220,-0.129744,-0.192545
4,in_sample,2022-04-01,2023-09-30,8,96,8,-0.037855,0.068762,-0.073389,0.501704,-0.515816,412 days,0.470803,0.205503,0.837255,1711.0,-0.001486,0.063056,-0.038673,-0.016453,0.160644,-0.331212,0.161775,-0.650231,-0.439221,0.767062,0.129684,0.124595,0.124733,0.131273,0.144978,-0.180106,-0.084208,-0.169411,-0.118638,-0.188071
21,in_sample,2022-04-01,2023-09-30,14,56,8,-0.039257,0.062649,-0.075558,0.497588,-0.519565,412 days,0.476277,0.198877,0.842433,1750.0,-0.000879,0.078131,-0.052672,-0.010531,0.156591,-0.323202,0.276250,-0.794777,-0.380872,0.742655,0.130602,0.124525,0.121166,0.134365,0.145012,-0.182756,-0.080386,-0.168683,-0.117348,-0.183156
9,in_sample,2022-04-01,2023-09-30,10,80,8,-0.045064,0.052496,-0.087490,0.494756,-0.515081,412 days,0.474453,0.185860,0.852624,1697.0,-0.001744,0.064805,-0.047338,-0.014057,0.158350,-0.333247,0.175350,-0.732035,-0.415447,0.750994,0.129678,0.124089,0.123454,0.132480,0.145531,-0.181506,-0.080794,-0.169581,-0.116878,-0.186173


In [127]:
df_performance_1[df_performance_1.sampling_category == 'out_sample'].sort_values('annualized_sharpe_ratio', ascending=False)

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
49,out_sample,2023-10-01,2024-03-31,20,320,8,0.726777,1.254068,2.079209,0.505519,-0.349545,96 days,0.530055,0.95794,0.339364,579.0,0.257288,-0.010984,0.33402,0.182476,0.123521,1.334478,-0.356197,1.702805,0.861648,0.586691,0.145391,0.143287,0.147355,0.15154,0.132156,-0.08878,-0.100088,-0.107263,-0.098611,-0.123269


In [189]:
df_performance_2[df_performance_2.sampling_category == 'in_sample'].sort_values('annualized_sharpe_ratio', ascending=False)

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
48,in_sample,2022-10-01,2024-03-31,20,320,8,0.686419,1.280979,2.209764,0.455635,-0.310630,207 days,0.507299,1.702940,0.089147,1620.0,0.179043,0.152945,0.121072,0.103956,0.175375,1.019979,0.853647,0.579414,0.477589,0.984986,0.122247,0.119090,0.128112,0.121400,0.123170,-0.102480,-0.071084,-0.162665,-0.092119,-0.091259
47,in_sample,2022-10-01,2024-03-31,20,280,8,0.687558,1.276735,2.297285,0.457232,-0.299291,206 days,0.500000,1.696828,0.090298,1631.0,0.178381,0.151048,0.123733,0.102047,0.179021,1.000284,0.831116,0.602950,0.462365,1.025058,0.124341,0.120515,0.126809,0.121684,0.121187,-0.103079,-0.077168,-0.149338,-0.091377,-0.092313
46,in_sample,2022-10-01,2024-03-31,20,240,8,0.685918,1.270863,2.496618,0.460940,-0.274739,205 days,0.503650,1.689022,0.091785,1654.0,0.182270,0.154731,0.116798,0.094134,0.174020,1.007489,0.841143,0.556428,0.402620,1.016624,0.127102,0.123325,0.125998,0.121606,0.117569,-0.095495,-0.077263,-0.141121,-0.095687,-0.092772
41,in_sample,2022-10-01,2024-03-31,18,288,8,0.679897,1.266543,2.313975,0.457901,-0.293822,206 days,0.498175,1.684322,0.092690,1647.0,0.177615,0.149518,0.125955,0.101326,0.176802,0.991273,0.818254,0.614834,0.458461,1.011782,0.124873,0.120806,0.127805,0.121124,0.120862,-0.101579,-0.076341,-0.147199,-0.089478,-0.091408
40,in_sample,2022-10-01,2024-03-31,18,252,8,0.674200,1.256348,2.455943,0.459861,-0.274518,205 days,0.505474,1.671382,0.095218,1649.0,0.181625,0.152423,0.123516,0.093448,0.171086,1.001825,0.826331,0.599662,0.399390,0.996063,0.127294,0.123044,0.127275,0.120731,0.117445,-0.094173,-0.076250,-0.136598,-0.092772,-0.092044
34,in_sample,2022-10-01,2024-03-31,16,256,8,0.665208,1.244495,2.503982,0.460385,-0.265660,205 days,0.498175,1.656866,0.098120,1657.0,0.183342,0.154208,0.123048,0.088938,0.167697,1.008593,0.836284,0.593940,0.366073,0.974845,0.127983,0.123523,0.127929,0.120071,0.116939,-0.090781,-0.074351,-0.135579,-0.091551,-0.091640
39,in_sample,2022-10-01,2024-03-31,18,216,8,0.665063,1.243669,2.617562,0.461459,-0.254077,204 days,0.501825,1.655743,0.098347,1651.0,0.184874,0.159020,0.114111,0.085023,0.161581,1.013285,0.855573,0.534509,0.334722,0.953432,0.128780,0.126020,0.126452,0.120644,0.113612,-0.082383,-0.070631,-0.123627,-0.092397,-0.092488
45,in_sample,2022-10-01,2024-03-31,20,200,8,0.662105,1.238932,2.612108,0.461501,-0.253475,204 days,0.500000,1.649808,0.099556,1642.0,0.185951,0.160488,0.106682,0.087172,0.160668,1.021107,0.858917,0.484948,0.349607,0.951172,0.128683,0.127192,0.125153,0.121568,0.112972,-0.081560,-0.072846,-0.125514,-0.092738,-0.092860
33,in_sample,2022-10-01,2024-03-31,16,224,8,0.651985,1.227622,2.551269,0.460833,-0.255553,204 days,0.500000,1.636314,0.102349,1658.0,0.183840,0.157130,0.116498,0.081808,0.160471,1.003647,0.844382,0.548615,0.311129,0.944496,0.129188,0.125684,0.127324,0.119810,0.113670,-0.080329,-0.070426,-0.119052,-0.091094,-0.092250
27,in_sample,2022-10-01,2024-03-31,14,224,8,0.635484,1.207396,2.463220,0.460475,-0.257989,144 days,0.498175,1.611881,0.107565,1668.0,0.181042,0.158602,0.119690,0.076906,0.155568,0.978727,0.851592,0.567625,0.273564,0.914847,0.130156,0.126206,0.128347,0.119182,0.112544,-0.076875,-0.069945,-0.110140,-0.088107,-0.092198


In [191]:
df_performance_3[df_performance_3.sampling_category == 'in_sample'].sort_values('annualized_sharpe_ratio', ascending=False)

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
45,in_sample,2023-04-01,2024-09-30,20,200,8,0.611730,1.169466,2.699069,0.461949,-0.226645,147 days,0.520947,1.565696,0.117997,1594.0,0.026693,0.078981,0.113881,0.149352,0.231190,-0.129111,0.269171,0.564909,0.806195,1.380397,0.120531,0.136729,0.116668,0.121727,0.121719,-0.076653,-0.129550,-0.085458,-0.077986,-0.059088
46,in_sample,2023-04-01,2024-09-30,20,240,8,0.610184,1.166325,2.703761,0.462986,-0.225680,204 days,0.535519,1.561636,0.118951,1576.0,0.029156,0.069388,0.124556,0.142617,0.238785,-0.114138,0.204954,0.639912,0.756942,1.402221,0.117850,0.133137,0.118199,0.122197,0.124493,-0.080361,-0.129772,-0.085976,-0.091539,-0.058028
39,in_sample,2023-04-01,2024-09-30,18,216,8,0.606537,1.162280,2.707438,0.461556,-0.224026,147 days,0.520947,1.556863,0.120080,1599.0,0.025793,0.077470,0.118520,0.149317,0.231442,-0.137372,0.259851,0.594352,0.810011,1.377876,0.120107,0.135571,0.118190,0.120998,0.122127,-0.076781,-0.129732,-0.082433,-0.074461,-0.059402
40,in_sample,2023-04-01,2024-09-30,18,252,8,0.606668,1.161973,2.744348,0.462128,-0.221061,147 days,0.528233,1.556384,0.120194,1591.0,0.029196,0.068722,0.127846,0.143120,0.239417,-0.113000,0.200377,0.659785,0.764973,1.402212,0.118265,0.132817,0.119262,0.121354,0.124895,-0.079709,-0.129968,-0.080627,-0.087724,-0.058485
34,in_sample,2023-04-01,2024-09-30,16,256,8,0.602830,1.158022,2.770056,0.461629,-0.217624,147 days,0.522769,1.551784,0.121291,1602.0,0.027475,0.068887,0.131844,0.140433,0.236367,-0.125932,0.201421,0.687009,0.751783,1.386709,0.118938,0.133074,0.119781,0.120136,0.124459,-0.078711,-0.130144,-0.074827,-0.082827,-0.059231
33,in_sample,2023-04-01,2024-09-30,16,224,8,0.599426,1.152983,2.739150,0.461267,-0.218836,147 days,0.522769,1.545553,0.122790,1595.0,0.023899,0.077109,0.124749,0.147272,0.228406,-0.151707,0.257680,0.636656,0.798599,1.357656,0.120654,0.135209,0.119268,0.120390,0.122109,-0.076134,-0.129800,-0.074796,-0.073558,-0.059933
27,in_sample,2023-04-01,2024-09-30,14,224,8,0.584313,1.132987,2.716364,0.461885,-0.215109,146 days,0.522769,1.521214,0.128783,1588.0,0.022386,0.078950,0.124311,0.144617,0.223238,-0.161034,0.270042,0.630039,0.781178,1.329437,0.121978,0.135612,0.120045,0.120038,0.121437,-0.079122,-0.129140,-0.067993,-0.073430,-0.059904
41,in_sample,2023-04-01,2024-09-30,18,288,8,0.582962,1.130368,2.370366,0.461765,-0.245938,204 days,0.522769,1.517821,0.129636,1558.0,0.027462,0.061689,0.124668,0.140333,0.243514,-0.130008,0.150430,0.637548,0.744131,1.399201,0.117025,0.130436,0.119252,0.121465,0.127804,-0.085872,-0.130676,-0.086201,-0.091394,-0.057864
47,in_sample,2023-04-01,2024-09-30,20,280,8,0.568354,1.109356,2.221396,0.462726,-0.255854,205 days,0.522769,1.491925,0.136294,1553.0,0.024843,0.059144,0.120589,0.139303,0.241218,-0.153277,0.131813,0.609069,0.731743,1.380862,0.116502,0.130121,0.118742,0.122444,0.128185,-0.088765,-0.130775,-0.091455,-0.093582,-0.057419
38,in_sample,2023-04-01,2024-09-30,18,180,8,0.550069,1.085638,2.464328,0.460752,-0.223213,147 days,0.519126,1.463315,0.143954,1581.0,0.021997,0.086222,0.096920,0.137809,0.218578,-0.159493,0.314979,0.433265,0.722325,1.314227,0.124025,0.139539,0.116652,0.121843,0.120091,-0.084380,-0.128793,-0.078158,-0.079406,-0.059263


In [197]:
df_performance_4[df_performance_4.sampling_category == 'in_sample'].sort_values('annualized_sharpe_ratio', ascending=False)

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
46,in_sample,2023-10-01,2025-03-31,20,240,8,0.854930,1.389391,2.474821,0.508019,-0.345451,115 days,0.527372,1.821960,0.069007,1611.0,0.125494,0.095068,0.093179,0.211554,0.228156,0.587724,0.365569,0.377275,1.005217,1.220634,0.134816,0.144288,0.129659,0.154955,0.137228,-0.079348,-0.127733,-0.164306,-0.113742,-0.093942
47,in_sample,2023-10-01,2025-03-31,20,280,8,0.840544,1.373143,2.352602,0.512065,-0.357283,120 days,0.529197,1.801956,0.072103,1591.0,0.130023,0.082095,0.098818,0.207964,0.228246,0.618643,0.282370,0.416347,0.989159,1.216337,0.134599,0.144148,0.130037,0.154350,0.137843,-0.083115,-0.132316,-0.172362,-0.118441,-0.097445
45,in_sample,2023-10-01,2025-03-31,20,200,8,0.832576,1.362523,2.484184,0.511136,-0.335151,112 days,0.534672,1.788696,0.074217,1613.0,0.118698,0.110139,0.078494,0.213696,0.227890,0.535432,0.457202,0.272054,1.010122,1.221075,0.137106,0.146101,0.129882,0.156153,0.136968,-0.077325,-0.126626,-0.156795,-0.106985,-0.086814
40,in_sample,2023-10-01,2025-03-31,18,252,8,0.816008,1.346945,2.342477,0.508681,-0.348352,117 days,0.531022,1.769955,0.077292,1598.0,0.121140,0.094257,0.083259,0.208034,0.222596,0.556807,0.360270,0.304281,0.985801,1.185321,0.135390,0.144361,0.131336,0.155061,0.137480,-0.081133,-0.128369,-0.172867,-0.112397,-0.093899
41,in_sample,2023-10-01,2025-03-31,18,288,8,0.814495,1.345935,2.276768,0.509539,-0.357742,120 days,0.529197,1.768806,0.077483,1597.0,0.129460,0.083931,0.090757,0.202791,0.221270,0.613256,0.294200,0.357546,0.961072,1.173967,0.135057,0.144181,0.131083,0.154386,0.137929,-0.082531,-0.132216,-0.178099,-0.116741,-0.096577
39,in_sample,2023-10-01,2025-03-31,18,216,8,0.793450,1.320559,2.333889,0.509885,-0.339969,112 days,0.534672,1.737439,0.082873,1612.0,0.116390,0.104274,0.075723,0.205563,0.221940,0.519794,0.422119,0.251525,0.969232,1.184561,0.137243,0.145309,0.130507,0.155703,0.137060,-0.079115,-0.127064,-0.163614,-0.106953,-0.087345
48,in_sample,2023-10-01,2025-03-31,20,320,8,0.790472,1.319368,2.168390,0.508232,-0.364543,124 days,0.525547,1.736265,0.083080,1573.0,0.132120,0.072431,0.093083,0.188656,0.218143,0.632912,0.219282,0.375523,0.890614,1.154606,0.134508,0.144796,0.130547,0.153444,0.138015,-0.086081,-0.140561,-0.192397,-0.122247,-0.102147
34,in_sample,2023-10-01,2025-03-31,16,256,8,0.771417,1.297442,2.201773,0.508237,-0.350362,117 days,0.534672,1.709347,0.087954,1619.0,0.117416,0.093442,0.077961,0.199396,0.217534,0.529877,0.354772,0.266156,0.939696,1.154137,0.136092,0.144548,0.131864,0.154993,0.137575,-0.081463,-0.130378,-0.175120,-0.110254,-0.091861
33,in_sample,2023-10-01,2025-03-31,16,224,8,0.749079,1.269986,2.192569,0.510675,-0.341644,112 days,0.529197,1.675402,0.094427,1630.0,0.112432,0.101041,0.070997,0.197546,0.215262,0.492766,0.401820,0.217288,0.926333,1.142101,0.137561,0.145268,0.130866,0.155678,0.137345,-0.080838,-0.127348,-0.167346,-0.105329,-0.089897
38,in_sample,2023-10-01,2025-03-31,18,180,8,0.738426,1.254277,2.219575,0.512850,-0.332688,112 days,0.520073,1.655622,0.098372,1648.0,0.109680,0.115122,0.062552,0.192654,0.217335,0.469024,0.482812,0.156583,0.895139,1.159991,0.139844,0.148399,0.129750,0.156695,0.137004,-0.088540,-0.126539,-0.159057,-0.103721,-0.090898


In [ ]:
df_performance_5

In [250]:
df_performance = pd.concat([df_performance_1, df_performance_2, df_performance_3, df_performance_4], axis=0, ignore_index=True)

In [252]:
df_performance.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Walk_Forward_Performance-2022-04-01-2025-07-31.pickle')

## Analyze Walk Forward Performance Results

In [254]:
df_performance = pd.read_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Walk_Forward_Performance-2021-06-01-2025-06-30.pickle')

In [256]:
df_performance = pd.concat([df_performance_1, df_performance_2, df_performance_3, df_performance_4], axis=0, ignore_index=True)

In [258]:
out_of_sample_cond = (df_performance['sampling_category'] == 'out_sample')
df_performance_os = df_performance[out_of_sample_cond]
df_performance_is = df_performance[~out_of_sample_cond]

In [260]:
## In-Sample
df_performance_is['mavg_strategy'] = 'f' + df_performance_is['fast_mavg'].astype(str) + '_s' + df_performance_is['slow_mavg'].astype(str) + '_n' + df_performance_is['mavg_stepsize'].astype(str)
df_performance_is['strategy_fold'] = (pd.to_datetime(df_performance_is['start_date']).dt.strftime("%Y-%m") + " → " + pd.to_datetime(df_performance_is['end_date']).dt.strftime("%Y-%m"))
remove_strategy_fold_is = (df_performance_is['strategy_fold'] != '2023-06 → 2025-05')
df_performance_is = df_performance_is[remove_strategy_fold_is]

## Out of Sample
df_performance_os['mavg_strategy'] = 'f' + df_performance_os['fast_mavg'].astype(str) + '_s' + df_performance_os['slow_mavg'].astype(str) + '_n' + df_performance_os['mavg_stepsize'].astype(str)
df_performance_os['strategy_fold'] = (pd.to_datetime(df_performance_os['start_date']).dt.strftime("%Y-%m") + " → " + pd.to_datetime(df_performance_os['end_date']).dt.strftime("%Y-%m"))
remove_strategy_fold_os = (df_performance_os['strategy_fold'] != '2025-06 → 2025-11')
df_performance_os = df_performance_os[remove_strategy_fold_os]

## Calculate Average Sharpe Ratio across all folds
# sharpe_cols = [col for col in df_performance_os.columns if 'sharpe' in col]
# std_dev_cols = [col for col in df_performance_os.columns if 'std_dev' in col]
# df_performance_os['sharpe_ratio_mean'] = df_performance_os[sharpe_cols].mean()
# df_performance_os['std_dev_mean'] = df_performance_os[std_dev_cols].mean()

In [262]:
df_performance_is.groupby(['strategy_fold']).size()

strategy_fold
2022-04 → 2023-09    49
2022-10 → 2024-03    49
2023-04 → 2024-09    49
2023-10 → 2025-03    49
dtype: int64

In [264]:
df_performance_os.groupby(['strategy_fold']).size()

strategy_fold
2023-10 → 2024-03    1
2024-04 → 2024-09    1
2024-10 → 2025-03    1
2025-04 → 2025-07    1
dtype: int64

In [266]:
## In Sample Strategy Analysis
strategy_aggs_dict = {'annualized_sharpe_ratio': 'max',
                      'annualized_std_dev': 'max',
                      'BTC-USD_annualized_sharpe_ratio': 'max',
                      'ETH-USD_annualized_sharpe_ratio': 'max',
                      'SOL-USD_annualized_sharpe_ratio': 'max',
                      'ADA-USD_annualized_sharpe_ratio': 'max',
                      'AVAX-USD_annualized_sharpe_ratio': 'max'}
df_mavg_strategy_results_is = pd.pivot_table(df_performance_is, index='mavg_strategy', columns='strategy_fold',
                                             values=['annualized_sharpe_ratio','annualized_std_dev','BTC-USD_annualized_sharpe_ratio','ETH-USD_annualized_sharpe_ratio','SOL-USD_annualized_sharpe_ratio','ADA-USD_annualized_sharpe_ratio','AVAX-USD_annualized_sharpe_ratio'],
                                             aggfunc=strategy_aggs_dict)
for col in df_mavg_strategy_results_is['annualized_sharpe_ratio'].columns:
    df_mavg_strategy_results_is[f'{col}_rank'] = df_mavg_strategy_results_is['annualized_sharpe_ratio'][col].rank(ascending=False)

rank_cols = [f'{col}_rank' for col in df_mavg_strategy_results_is['annualized_sharpe_ratio'].columns]
df_mavg_strategy_results_is['top_5_rank_count'] = (df_mavg_strategy_results_is[rank_cols] <= 5).sum(axis=1)
df_mavg_strategy_results_is['strategy_avg_rank'] = df_mavg_strategy_results_is[rank_cols].sum(axis=1)/5
sharpe_cols = df_mavg_strategy_results_is['annualized_sharpe_ratio'].columns
df_mavg_strategy_results_is['sharpe_mean_is'] = df_mavg_strategy_results_is['annualized_sharpe_ratio'].mean(axis=1)
df_mavg_strategy_results_is['std_dev_mean_is'] = df_mavg_strategy_results_is['annualized_std_dev'].mean(axis=1)

In [268]:
df_mavg_strategy_results_os = pd.pivot_table(df_performance_os, index='mavg_strategy', columns='strategy_fold',
                                             values=['annualized_sharpe_ratio','annualized_std_dev','BTC-USD_annualized_sharpe_ratio','ETH-USD_annualized_sharpe_ratio','SOL-USD_annualized_sharpe_ratio','ADA-USD_annualized_sharpe_ratio','AVAX-USD_annualized_sharpe_ratio'],
                                             aggfunc=strategy_aggs_dict)
for col in df_mavg_strategy_results_os['annualized_sharpe_ratio'].columns:
    df_mavg_strategy_results_os[f'{col}_rank'] = df_mavg_strategy_results_os['annualized_sharpe_ratio'][col].rank(ascending=False)

rank_cols = [f'{col}_rank' for col in df_mavg_strategy_results_os['annualized_sharpe_ratio'].columns]
df_mavg_strategy_results_os['top_5_rank_count'] = (df_mavg_strategy_results_os[rank_cols] <= 5).sum(axis=1)
df_mavg_strategy_results_os['strategy_avg_rank'] = df_mavg_strategy_results_os[rank_cols].sum(axis=1)/5
sharpe_cols = df_mavg_strategy_results_os['annualized_sharpe_ratio'].columns
df_mavg_strategy_results_os['sharpe_mean_os'] = df_mavg_strategy_results_os['annualized_sharpe_ratio'].mean(axis=1)
df_mavg_strategy_results_os['std_dev_mean_os'] = df_mavg_strategy_results_os['annualized_std_dev'].mean(axis=1)

In [270]:
rank_cond = (df_mavg_strategy_results_os['top_5_rank_count'] > 0)
df_top_strategy_performance_os = df_mavg_strategy_results_os[rank_cond].sort_values(['sharpe_mean_os'], ascending=[False])
df_top_strategy_performance_os

ADA-USD_annualized_sharpe_ratio                    \
strategy_fold               2023-10 → 2024-03 2024-04 → 2024-09   
mavg_strategy                                                     
f10_s40_n8                           0.861648               NaN   
f20_s200_n8                               NaN               NaN   
f20_s320_n8                               NaN          0.795056   
f20_s240_n8                               NaN               NaN   

                                                   \
strategy_fold 2024-10 → 2025-03 2025-04 → 2025-07   
mavg_strategy                                       
f10_s40_n8                  NaN               NaN   
f20_s200_n8            0.840731               NaN   
f20_s320_n8                 NaN               NaN   
f20_s240_n8                 NaN         -0.907992   

              AVAX-USD_annualized_sharpe_ratio                    \
strategy_fold                2023-10 → 2024-03 2024-04 → 2024-09   
mavg_strategy                                                      
f10_s40_n8                            0.586691               NaN   
f20_s200_n8                                NaN               NaN   
f20_s320_n8                                NaN          0.135208   
f20_s240_n8                                NaN               NaN   

                                                   \
strategy_fold 2024-10 → 2025-03 2025-04 → 2025-07   
mavg_strategy                                       
f10_s40_n8                  NaN               NaN   
f20_s200_n8            1.441356               NaN   
f20_s320_n8                 NaN               NaN   
f20_s240_n8                 NaN         -4.945377   

              BTC-USD_annualized_sharpe_ratio                    \
strategy_fold               2023-10 → 2024-03 2024-04 → 2024-09   
mavg_strategy                                                     
f10_s40_n8                           1.334478               NaN   
f20_s200_n8                               NaN               NaN   
f20_s320_n8                               NaN          -1.38316   
f20_s240_n8                               NaN               NaN   

                                                   \
strategy_fold 2024-10 → 2025-03 2025-04 → 2025-07   
mavg_strategy                                       
f10_s40_n8                  NaN               NaN   
f20_s200_n8            1.120584               NaN   
f20_s320_n8                 NaN               NaN   
f20_s240_n8                 NaN         -0.827782   

              ETH-USD_annualized_sharpe_ratio                    \
strategy_fold               2023-10 → 2024-03 2024-04 → 2024-09   
mavg_strategy                                                     
f10_s40_n8                          -0.356197               NaN   
f20_s200_n8                               NaN               NaN   
f20_s320_n8                               NaN          -0.92326   
f20_s240_n8                               NaN               NaN   

                                                   \
strategy_fold 2024-10 → 2025-03 2025-04 → 2025-07   
mavg_strategy                                       
f10_s40_n8                  NaN               NaN   
f20_s200_n8            0.192144               NaN   
f20_s320_n8                 NaN               NaN   
f20_s240_n8                 NaN          -0.91767   

              SOL-USD_annualized_sharpe_ratio                    \
strategy_fold               2023-10 → 2024-03 2024-04 → 2024-09   
mavg_strategy                                                     
f10_s40_n8                           1.702805               NaN   
f20_s200_n8                               NaN               NaN   
f20_s320_n8                               NaN         -1.314647   
f20_s240_n8                               NaN               NaN   

                                                  annualized_sharpe_ratio  \
strategy_fold 2024-10 → 2025-03 2025-04 → 2025-07       2023-10 → 2024-03   
mavg_strategy  

In [272]:
df_top_strategy_performance_os.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Top_Moving_Average_Ribbon_Portfolio_Performance_Out_Of_Sample-2022-04-01-2025-07-31.pickle')

In [274]:
df_mavg_strategy_results_is.loc[df_top_strategy_performance_os.index.tolist()]

ADA-USD_annualized_sharpe_ratio                    \
strategy_fold               2022-04 → 2023-09 2022-10 → 2024-03   
mavg_strategy                                                     
f10_s40_n8                          -0.316034          0.768347   
f20_s200_n8                         -1.007143          0.349607   
f20_s320_n8                         -1.098521          0.477589   
f20_s240_n8                         -1.067760          0.402620   

                                                   \
strategy_fold 2023-04 → 2024-09 2023-10 → 2025-03   
mavg_strategy                                       
f10_s40_n8             0.214400         -0.044883   
f20_s200_n8            0.806195          1.010122   
f20_s320_n8            0.726114          0.890614   
f20_s240_n8            0.756942          1.005217   

              AVAX-USD_annualized_sharpe_ratio                    \
strategy_fold                2022-04 → 2023-09 2022-10 → 2024-03   
mavg_strategy                                                      
f10_s40_n8                            0.645827          0.363694   
f20_s200_n8                           0.640101          0.951172   
f20_s320_n8                           0.488875          0.984986   
f20_s240_n8                           0.612736          1.016624   

                                                   \
strategy_fold 2023-04 → 2024-09 2023-10 → 2025-03   
mavg_strategy                                       
f10_s40_n8             0.521907          0.548714   
f20_s200_n8            1.380397          1.221075   
f20_s320_n8            1.327693          1.154606   
f20_s240_n8            1.402221          1.220634   

              BTC-USD_annualized_sharpe_ratio                    \
strategy_fold               2022-04 → 2023-09 2022-10 → 2024-03   
mavg_strategy                                                     
f10_s40_n8                          -0.110789          0.850019   
f20_s200_n8                         -0.259248          1.021107   
f20_s320_n8                         -0.118828          1.019979   
f20_s240_n8                         -0.217830          1.007489   

                                                   \
strategy_fold 2023-04 → 2024-09 2023-10 → 2025-03   
mavg_strategy                                       
f10_s40_n8            -0.175970          0.452664   
f20_s200_n8           -0.129111          0.535432   
f20_s320_n8           -0.204607          0.632912   
f20_s240_n8           -0.114138          0.587724   

              ETH-USD_annualized_sharpe_ratio                    \
strategy_fold               2022-04 → 2023-09 2022-10 → 2024-03   
mavg_strategy                                                     
f10_s40_n8                           0.444382          0.033998   
f20_s200_n8                         -0.039606          0.858917   
f20_s320_n8                         -0.087287          0.853647   
f20_s240_n8                         -0.087470          0.841143   

                                                   \
strategy_fold 2023-04 → 2024-09 2023-10 → 2025-03   
mavg_strategy                                       
f10_s40_n8            -0.300133          0.190703   
f20_s200_n8            0.269171          0.457202   
f20_s320_n8            0.051205          0.219282   
f20_s240_n8            0.204954          0.365569   

              SOL-USD_annualized_sharpe_ratio                    \
strategy_fold               2022-04 → 2023-09 2022-10 → 2024-03   
mavg_strategy                                                     
f10_s40_n8                          -0.663305          0.383101   
f20_s200_n8                         -0.647258          0.484948   
f20_s320_n8                         -0.792715          0.579414   
f20_s240_n8                         -0.691763          0.556428   

                                                  annualized_sharpe_ratio  \
strategy_fold 2023-04 → 2024-09 2023-10 → 2025-03       2022-04 → 2023-09   
mavg_strategy  

In [ ]:
df_performance_os.head()

In [ ]:
df_performance_os#.sort_values(['sampling_category','fast_mavg','slow_mavg','mavg_stepsize','start_date'])

In [ ]:
df_performance.groupby(['sampling_category']).size()

In [ ]:
df_performance[out_of_sample_cond].sort_values('annualized_sharpe_ratio', ascending=False)

## Re-Run the Walk Forward Analysis for Locked Pairs

In [660]:
import itertools

def generate_moving_avg_ribbon_loecked_pairs_params():
    parameter_grid = {
        "fast_window": [10, 20],
        "slow_fast_ratio":[4, 8, 10, 12],
        "stepsize":[8],
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [358]:
def run_walk_forward_moving_avg_ribbon_locked_pairs(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'fast_mavg', 'slow_mavg', 'mavg_stepsize', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        # print(f'In Sample Start: {start_date_is}, In Sample End: {end_date_is}, Out of Sample Start: {start_date_os}, Out of Sample End: {end_date_os}')
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        for params in generate_moving_avg_ribbon_loecked_pairs_params():
            print(params)
            fast_mavg = params['fast_window']
            slow_mavg = params['slow_fast_ratio'] * fast_mavg
            mavg_stepsize = params['stepsize']
            print(fast_mavg, slow_mavg, mavg_stepsize)
            
            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = apply_target_volatility_position_sizing_continuous_strategy(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
                use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
                annualized_target_volatility=annualized_target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= start_date_is]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = calculate_asset_level_returns(df_is, end_date, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'fast_mavg': fast_mavg,
                'slow_mavg': slow_mavg,
                'mavg_stepsize': mavg_stepsize
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)

            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample and out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        ## Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        best_in_sample_fast_mavg = df_performance[in_sample_cond & date_cond].sort_values('annualized_sharpe_ratio', ascending=False)['fast_mavg'].iloc[0]
        best_in_sample_slow_mavg = df_performance[in_sample_cond & date_cond].sort_values('annualized_sharpe_ratio', ascending=False)['slow_mavg'].iloc[0]
        print(f'Best In Sample Fast Mavg: {best_in_sample_fast_mavg}')
        print(f'Best In Sample Slow Mavg: {best_in_sample_slow_mavg}')

        ## Out of Sample Dataframe
        df_os = apply_target_volatility_position_sizing_continuous_strategy(
            start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=best_in_sample_fast_mavg, slow_mavg=best_in_sample_slow_mavg,
            mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
            use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
            initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
            rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
            transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
            rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
            annualized_target_volatility=annualized_target_volatility,
            annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_os)

        df_os = df_os[df_os.index >= start_date_os]
        print('Calculating Out of Sample Asset Returns!!')
        df_os = calculate_asset_level_returns(df_os, end_date, ticker_list)

        ## Out of Sample Performance Metrics
        print('Pulling Out of Sample Performance Metrics!!')
        row_parameters_os = {
            'sampling_category': 'out_sample',
            'start_date': start_date_os,
            'end_date': end_date_os,
            'fast_mavg': best_in_sample_fast_mavg,
            'slow_mavg': best_in_sample_slow_mavg,
            'mavg_stepsize': mavg_stepsize
        }
        portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                           strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                           passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

        print('Getting Out of Sample Asset Performance!!')
        for ticker in ticker_list:
            ## Out of Sample
            ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                 strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                 annual_trading_days=365, include_transaction_costs_and_fees=False)
            ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
            ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
            portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
        
        row_parameters_os.update(portfolio_perf_metrics_os)

        ## Assign in sample and out of sample metrics to performance dataframe
        df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()

    return df_performance

In [283]:
%%time
df_performance_locked_1 = run_walk_forward_moving_avg_ribbon_locked_pairs(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'fast_window': 10, 'slow_fast_ratio': 4, 'stepsize': 8}
10 40 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 10, 'slow_fast_ratio': 8, 'stepsize': 8}
10 80 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Vola

In [285]:
df_performance_locked_1.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Locked_Pairs-2022-04-01-2024-04-01.pickle')

In [287]:
%%time
df_performance_locked_2 = run_walk_forward_moving_avg_ribbon_locked_pairs(start_date='2022-10-01', end_date='2024-10-01', ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'fast_window': 10, 'slow_fast_ratio': 4, 'stepsize': 8}
10 40 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 10, 'slow_fast_ratio': 8, 'stepsize': 8}
10 80 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Vola

In [288]:
df_performance_locked_2.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Locked_Pairs-2022-10-01-2024-10-01.pickle')

In [289]:
%%time
df_performance_locked_3 = run_walk_forward_moving_avg_ribbon_locked_pairs(start_date='2023-04-01', end_date='2025-04-01', ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'fast_window': 10, 'slow_fast_ratio': 4, 'stepsize': 8}
10 40 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 10, 'slow_fast_ratio': 8, 'stepsize': 8}
10 80 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Vola

In [290]:
df_performance_locked_3.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Locked_Pairs-2023-04-01-2025-04-01.pickle')

In [291]:
%%time
df_performance_locked_4 = run_walk_forward_moving_avg_ribbon_locked_pairs(start_date='2023-10-01', end_date='2025-10-01', ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'fast_window': 10, 'slow_fast_ratio': 4, 'stepsize': 8}
10 40 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_window': 10, 'slow_fast_ratio': 8, 'stepsize': 8}
10 80 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop L

In [292]:
df_performance_locked_4.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Locked_Pairs-2023-10-01-2025-10-01.pickle')

In [295]:
df_performance_locked = pd.concat([df_performance_locked_1, df_performance_locked_2, df_performance_locked_3, df_performance_locked_4], axis=0, ignore_index=True)

In [303]:
df_performance_locked.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Locked_Pairs-2022-04-01-2025-10-01.pickle')

In [305]:
df_performance_locked

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,10,40,8,0.014463,0.173555,0.027254,0.491599,-0.530680,412 days,0.474453,0.334610,0.738048,1776.0,0.026189,0.101508,-0.037694,-0.000641,0.135580,-0.110789,0.444382,-0.663305,-0.316034,0.645827,0.132792,0.126587,0.121675,0.130595,0.136379,-0.159788,-0.097625,-0.172257,-0.137484,-0.179451
1,in_sample,2022-04-01,2023-09-30,10,80,8,-0.045064,0.052496,-0.087490,0.494756,-0.515081,412 days,0.474453,0.185860,0.852624,1697.0,-0.001744,0.064805,-0.047338,-0.014057,0.158350,-0.333247,0.175350,-0.732035,-0.415447,0.750994,0.129678,0.124089,0.123454,0.132480,0.145531,-0.181506,-0.080794,-0.169581,-0.116878,-0.186173
2,in_sample,2022-04-01,2023-09-30,10,100,8,-0.063661,0.013715,-0.125275,0.501545,-0.508174,468 days,0.478102,0.138077,0.890230,1712.0,-0.008320,0.051047,-0.042380,-0.030099,0.161779,-0.385118,0.069068,-0.682788,-0.542786,0.756710,0.129680,0.123649,0.124515,0.132086,0.149015,-0.186561,-0.086307,-0.161997,-0.122080,-0.179114
3,in_sample,2022-04-01,2023-09-30,10,120,8,-0.079566,-0.020117,-0.152118,0.502589,-0.523055,468 days,0.485401,0.096420,0.923222,1709.0,-0.014045,0.039594,-0.027789,-0.043620,0.157453,-0.427041,-0.023200,-0.554733,-0.646254,0.732509,0.130108,0.121773,0.125590,0.132785,0.148760,-0.198930,-0.096636,-0.153488,-0.129704,-0.175586
4,in_sample,2022-04-01,2023-09-30,20,80,8,-0.175602,-0.245887,-0.336614,0.501230,-0.521671,468 days,0.479927,-0.179739,0.857424,1694.0,-0.014814,0.011550,-0.070026,-0.058577,0.127213,-0.431658,-0.250358,-0.936195,-0.758979,0.530796,0.130074,0.122015,0.122548,0.133905,0.157620,-0.186784,-0.109091,-0.146219,-0.136101,-0.168339
5,in_sample,2022-04-01,2023-09-30,20,160,8,-0.102246,-0.060369,-0.178177,0.510545,-0.573846,468 days,0.476277,0.044921,0.964187,1699.0,-0.000737,0.041288,-0.048331,-0.068529,0.140792,-0.317514,-0.011244,-0.721563,-0.811619,0.643818,0.131528,0.120182,0.126207,0.137814,0.145992,-0.186275,-0.116657,-0.195234,-0.161906,-0.178921
6,in_sample,2022-04-01,2023-09-30,20,200,8,-0.120016,-0.104408,-0.205883,0.504893,-0.582934,468 days,0.478102,-0.008229,0.993437,1713.0,0.006743,0.038041,-0.041140,-0.092815,0.136619,-0.259248,-0.039606,-0.647258,-1.007143,0.640101,0.131772,0.118661,0.128542,0.138006,0.139691,-0.173823,-0.127845,-0.210354,-0.176903,-0.185172
7,in_sample,2022-04-01,2023-09-30,20,240,8,-0.144886,-0.164249,-0.248116,0.501975,-0.583946,468 days,0.476277,-0.081127,0.935370,1714.0,0.011839,0.032471,-0.046415,-0.101154,0.131014,-0.217830,-0.087470,-0.691763,-1.067760,0.612736,0.132736,0.117597,0.128692,0.138836,0.137269,-0.156862,-0.136188,-0.221725,-0.187988,-0.183924
8,out_sample,2023-10-01,2024-03-31,10,40,8,0.726777,1.254068,2.079209,0.505519,-0.349545,96 days,0.530055,0.957940,0.339364,579.0,0.257288,-0.010984,0.334020,0.182476,0.123521,1.334478,-0.356197,1.702805,0.861648,0.586691,0.145391,0.143287,0.147355,0.151540,0.132156,-0.088780,-0.100088,-0.107263,-0.098611,-0.123269
9,in_sample,2022-10-01,2024-03-31,10,40,8,0.375778,0.832030,1.201484,0.451692,-0.312762,130 days,0.494526,1.154712,0.248713,1717.0,0.160838,0.047024,0.091901,0.151811,0.088254,0.850019,0.033998,0.383101,0.768347,0.363694,0.129164,0.118032,0.122410,0.132244,0.118687,-0.072485,-0.087723,-0.098192,-0.102291,-0.109480


## Analyze Results of the Walk Forward Analysis for Locked Pairs

In [360]:
%%time
df_performance_locked_add = run_walk_forward_moving_avg_ribbon_locked_pairs(start_date='2022-04-01', end_date='2025-10-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'fast_window': 20, 'slow_fast_ratio': 8, 'stepsize': 8}
20 160 8
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Best In Sample Fast Mavg: 20
Best In Sample Slow Mavg: 160
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size 

In [362]:
df_performance_locked_add

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,20,160,8,-0.102246,-0.060369,-0.178177,0.510545,-0.573846,468 days,0.476277,0.044921,0.964187,1699.0,-0.000737,0.041288,-0.048331,-0.068529,0.140792,-0.317514,-0.011244,-0.721563,-0.811619,0.643818,0.131528,0.120182,0.126207,0.137814,0.145992,-0.186275,-0.116657,-0.195234,-0.161906,-0.178921
1,out_sample,2023-10-01,2024-03-31,20,160,8,1.738597,2.174014,6.744340,0.503896,-0.257786,96 days,0.513661,1.608714,0.109412,610.0,0.199211,0.281227,0.361174,0.267873,0.273828,0.907998,1.246362,2.172256,1.584137,1.733627,0.163605,0.173357,0.123328,0.124197,0.117559,-0.105570,-0.083125,-0.077600,-0.078877,-0.050546
2,in_sample,2022-10-01,2024-03-31,20,160,8,0.571111,1.116751,2.221848,0.460805,-0.257043,147 days,0.498175,1.500520,0.134057,1664.0,0.175610,0.167779,0.075045,0.081788,0.132068,0.926350,0.880327,0.253555,0.306804,0.752870,0.132853,0.131995,0.123324,0.122185,0.108952,-0.081786,-0.081503,-0.119404,-0.081814,-0.092229
3,out_sample,2024-04-01,2024-09-30,20,160,8,0.065887,0.277739,0.316650,0.502423,-0.208075,55 days,0.540984,0.266628,0.790058,506.0,-0.092442,-0.041718,-0.035690,0.157478,0.145930,-1.150308,-0.572903,-0.614846,0.835280,0.719515,0.121781,0.143506,0.126029,0.126474,0.135427,-0.061017,-0.082579,-0.062820,-0.055065,-0.058530
4,in_sample,2023-04-01,2024-09-30,20,160,8,0.528944,1.056440,2.345492,0.460734,-0.225515,147 days,0.517304,1.427748,0.153934,1578.0,0.021258,0.090502,0.084313,0.132067,0.217825,-0.162145,0.339721,0.338018,0.680592,1.315018,0.125443,0.142166,0.114576,0.121858,0.119489,-0.087528,-0.128296,-0.080751,-0.080933,-0.058897
5,out_sample,2024-10-01,2025-03-31,20,160,8,0.735133,1.235113,2.238865,0.526119,-0.328351,112 days,0.494505,0.939235,0.348862,540.0,0.234077,0.076175,-0.109277,0.210572,0.293849,1.235073,0.257652,-1.155822,0.819643,1.449714,0.140083,0.127813,0.137181,0.200379,0.153665,-0.069380,-0.070690,-0.152908,-0.104093,-0.091861
6,in_sample,2023-10-01,2025-03-31,20,160,8,0.731989,1.245614,2.205709,0.513153,-0.331861,112 days,0.518248,1.644809,0.100584,1651.0,0.108579,0.119772,0.058993,0.190041,0.223072,0.458786,0.506789,0.130557,0.881488,1.190002,0.141173,0.150184,0.128808,0.156611,0.137616,-0.091523,-0.126759,-0.156272,-0.104029,-0.090363
7,out_sample,2025-04-01,2025-07-31,20,160,8,-0.391728,-0.946326,-1.560026,0.466774,-0.251103,82 days,0.428571,-0.419961,0.675515,297.0,0.006474,0.062849,-0.060524,-0.108126,-0.211987,-0.310262,0.156549,-0.854085,-1.038835,-2.735656,0.115916,0.138118,0.122373,0.150135,0.103548,-0.056079,-0.051056,-0.057090,-0.083293,-0.076045


In [345]:
df_performance_locked_is = df_performance_locked[df_performance_locked.sampling_category == 'in_sample']
df_performance_locked_os = df_performance_locked[df_performance_locked.sampling_category == 'out_sample']

In [347]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_locked_is.groupby(['fast_mavg','slow_mavg']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                     median      mean       std   
fast_mavg slow_mavg                                               
10        40                       0.469941  0.486367  0.307246   
          80                       0.648455  0.534802  0.333616   
          100                      0.672112  0.530125  0.348356   
          120                      0.696390  0.535961  0.372571   
20        80                       0.449599  0.346947  0.418220   
          160                      1.086596  0.839609  0.605151   
          200                      1.204199  0.916628  0.685358   
          240                      1.218594  0.915583  0.725632   

                    annualized_return                     max_drawdown  \
                               median      mean       std       median   
fast_mavg slow_mavg                                                      
10        40                 0.182220  0.188671  0.169176    -0.517406   
          80                 0.268516  0.216542  0.184212    -0.393469   
          100                0.281948  0.214270  0.189424    -0.384649   
          120                0.296422  0.219225  0.201821    -0.380639   
20        80                 0.161778  0.121238  0.214847    -0.377228   
          160                0.550028  0.432450  0.367043    -0.294452   
          200                0.636918  0.496599  0.421799    -0.294313   
          240                0.648051  0.501537  0.442926    -0.310095   

                                        BTC-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40        -0.472593  0.107770                        0.170938   
          80        -0.391667  0.103504                        0.178800   
          100       -0.384493  0.104503                        0.190373   
          120       -0.391264  0.111139                        0.158404   
20        80        -0.394192  0.096988                        0.050071   
          160       -0.347066  0.157627                        0.148320   
          200       -0.349551  0.162287                        0.203160   
          240       -0.357454  0.158796                        0.236793   

                                        ETH-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         0.253981  0.487392                        0.112351   
          80         0.176116  0.491512                        0.212634   
          100        0.164856  0.499514                        0.339929   
          120        0.144164  0.515491                        0.382036   
20        80         0.051078  0.464762                        0.200669   
          160        0.226369  0.574672                        0.423255   
          200        0.292045  0.597792                        0.363187   
          240        0.315811  0.583662                        0.285261   

                                        SOL-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         0.092238  0.311474                        0.191874   
          80         0.193696  0.131837                        0.099853   
          100        0.302620  0.190747                        0.011484   
          120        0.310084  0.246758                        0.003039   
20        80         0.158447  0.309272                       -0.255463   
          160        0.428898  0.370375                        0.192056   
          200        0.386421  0.375725                        0.378501   
          240        0.331049  0.388349                        0.

In [349]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_locked_os.groupby(['fast_mavg','slow_mavg']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                     median      mean       std   
fast_mavg slow_mavg                                               
10        40                       1.254068  1.254068       NaN   
20        200                      1.179849  1.179849       NaN   
          240                     -0.873101 -0.873101  1.293271   

                    annualized_return                     max_drawdown  \
                               median      mean       std       median   
fast_mavg slow_mavg                                                      
10        40                 0.726777  0.726777       NaN    -0.349545   
20        200                0.679147  0.679147       NaN    -0.303374   
          240               -0.306935 -0.306935  0.362915    -0.258926   

                                        BTC-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40        -0.349545       NaN                        1.334478   
20        200       -0.303374       NaN                        1.120584   
          240       -0.258926  0.073424                       -1.065919   

                                        ETH-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         1.334478       NaN                       -0.356197   
20        200        1.120584       NaN                        0.192144   
          240       -1.065919  0.336777                       -0.861801   

                                       SOL-USD_annualized_sharpe_ratio  \
                         mean      std                          median   
fast_mavg slow_mavg                                                      
10        40        -0.356197      NaN                        1.702805   
20        200        0.192144      NaN                       -1.271911   
          240       -0.861801  0.07901                       -1.246547   

                                        ADA-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         1.702805       NaN                        0.861648   
20        200       -1.271911       NaN                        0.840731   
          240       -1.246547  0.435426                        0.013891   

                                       AVAX-USD_annualized_sharpe_ratio  \
                         mean      std                           median   
fast_mavg slow_mavg                                                       
10        40         0.861648      NaN                         0.586691   
20        200        0.840731      NaN                         1.441356   
          240        0.013891  1.30374                        -2.304297   

                                         
                         mean       std  
fast_mavg slow_mavg                      
10        40         0.586691       NaN  
20        200        1.441356       NaN  
          240       -2.304297  3.735051

In [370]:
df_performance_locked_is[df_performance_locked_is.fast_mavg == 20].sort_values('slow_mavg')

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
4,in_sample,2022-04-01,2023-09-30,20,80,8,-0.175602,-0.245887,-0.336614,0.501230,-0.521671,468 days,0.479927,-0.179739,0.857424,1694.0,-0.014814,0.011550,-0.070026,-0.058577,0.127213,-0.431658,-0.250358,-0.936195,-0.758979,0.530796,0.130074,0.122015,0.122548,0.133905,0.157620,-0.186784,-0.109091,-0.146219,-0.136101,-0.168339
13,in_sample,2022-10-01,2024-03-31,20,80,8,0.145582,0.417821,0.484240,0.451875,-0.300639,262 days,0.489051,0.647142,0.517812,1674.0,0.117518,0.078928,0.034053,0.039787,0.034019,0.535825,0.275001,-0.065386,-0.013909,-0.087258,0.134272,0.130890,0.122337,0.127990,0.109790,-0.087171,-0.089912,-0.107250,-0.109280,-0.104589
22,in_sample,2023-04-01,2024-09-30,20,80,8,0.177975,0.481377,0.522096,0.448061,-0.340886,279 days,0.489982,0.726282,0.467976,1653.0,0.009910,0.058291,0.025559,0.039202,0.142347,-0.249925,0.126337,-0.145575,-0.021086,0.783954,0.126236,0.143881,0.115984,0.124868,0.117509,-0.086278,-0.134408,-0.151496,-0.164448,-0.101985
31,in_sample,2023-10-01,2025-03-31,20,80,8,0.336998,0.734476,0.814849,0.504889,-0.413571,344 days,0.496350,1.020018,0.308171,1673.0,0.091991,0.116821,-0.008854,0.037626,0.180698,0.350068,0.482809,-0.365351,0.002743,0.955770,0.141584,0.152995,0.134367,0.156327,0.133403,-0.095709,-0.133219,-0.224566,-0.162105,-0.103589
5,in_sample,2022-04-01,2023-09-30,20,160,8,-0.102246,-0.060369,-0.178177,0.510545,-0.573846,468 days,0.476277,0.044921,0.964187,1699.0,-0.000737,0.041288,-0.048331,-0.068529,0.140792,-0.317514,-0.011244,-0.721563,-0.811619,0.643818,0.131528,0.120182,0.126207,0.137814,0.145992,-0.186275,-0.116657,-0.195234,-0.161906,-0.178921
14,in_sample,2022-10-01,2024-03-31,20,160,8,0.571111,1.116751,2.221848,0.460805,-0.257043,147 days,0.498175,1.500520,0.134057,1664.0,0.175610,0.167779,0.075045,0.081788,0.132068,0.926350,0.880327,0.253555,0.306804,0.752870,0.132853,0.131995,0.123324,0.122185,0.108952,-0.081786,-0.081503,-0.119404,-0.081814,-0.092229
23,in_sample,2023-04-01,2024-09-30,20,160,8,0.528944,1.056440,2.345492,0.460734,-0.225515,147 days,0.517304,1.427748,0.153934,1578.0,0.021258,0.090502,0.084313,0.132067,0.217825,-0.162145,0.339721,0.338018,0.680592,1.315018,0.125443,0.142166,0.114576,0.121858,0.119489,-0.087528,-0.128296,-0.080751,-0.080933,-0.058897
32,in_sample,2023-10-01,2025-03-31,20,160,8,0.731989,1.245614,2.205709,0.513153,-0.331861,112 days,0.518248,1.644809,0.100584,1651.0,0.108579,0.119772,0.058993,0.190041,0.223072,0.458786,0.506789,0.130557,0.881488,1.190002,0.141173,0.150184,0.128808,0.156611,0.137616,-0.091523,-0.126759,-0.156272,-0.104029,-0.090363
6,in_sample,2022-04-01,2023-09-30,20,200,8,-0.120016,-0.104408,-0.205883,0.504893,-0.582934,468 days,0.478102,-0.008229,0.993437,1713.0,0.006743,0.038041,-0.041140,-0.092815,0.136619,-0.259248,-0.039606,-0.647258,-1.007143,0.640101,0.131772,0.118661,0.128542,0.138006,0.139691,-0.173823,-0.127845,-0.210354,-0.176903,-0.185172
15,in_sample,2022-10-01,2024-03-31,20,200,8,0.662105,1.238932,2.612108,0.461501,-0.253475,204 days,0.500000,1.649808,0.099556,1642.0,0.185951,0.160488,0.106682,0.087172,0.160668,1.021107,0.858917,0.484948,0.349607,0.951172,0.128683,0.127192,0.125153,0.121568,0.112972,-0.081560,-0.072846,-0.125514,-0.092738,-0.092860


## Walk Forward Analysis of Equal Weighted Moving Average Ensemble of 10/40 and 20/200

In [570]:
nan_mask = (df_trend[rank_cols].notna().all(axis=1))
df_trend[f'{ticker}_mavg_ensemble_avg_rank'] = np.where(nan_mask, df_trend[rank_cols].mean(axis=1), np.nan)
df_trend[nan_mask].head()

,AVAX-USD_mavg_ribbon_slope_10_40,AVAX-USD_mavg_ribbon_rank_10_40,AVAX-USD_mavg_ribbon_slope_20_200,AVAX-USD_mavg_ribbon_rank_20_200,AVAX-USD_mavg_ensemble_avg_rank,AVAX-USD_close,AVAX-USD_open
date,,,,,,,
2022-08-05,0.044189,0.437333,-0.237075,0.089060,0.263196,24.95,23.37
2022-08-06,0.048676,0.440882,-0.230407,0.111223,0.276052,26.64,24.95
2022-08-07,0.057980,0.448642,-0.221813,0.138110,0.293376,27.90,26.65
2022-08-08,0.069166,0.456788,-0.212166,0.167347,0.312067,27.92,27.90
2022-08-09,0.077423,0.461479,-0.203182,0.194589,0.328034,27.43,27.91


In [572]:
(0.437333+0.089060)/2

0.2631965

In [576]:
## Original Signal
def generate_trend_signal_ensemble_with_donchian_channel_continuous(start_date, end_date, ticker, fast_mavg_pair, slow_mavg_pair, mavg_stepsize, mavg_z_score_window, entry_rolling_donchian_window, 
                                                                    exit_rolling_donchian_window, use_donchian_exit_gate, donchian_signal_weight, ma_crossover_signal_weight,
                                                                    use_activation=True, tanh_activation_constant_dict=None, 
                                                                    moving_avg_type='exponential', price_or_returns_calc='price',
                                                                    long_only=False, use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30'):

    # Pull Close Prices from Coinbase
    date_list = cn.coinbase_start_date_by_ticker_dict
    if use_saved_files:
        file_end_date = pd.Timestamp(saved_file_end_date).date()
        filename = f"{ticker}-pickle-{pd.Timestamp(date_list[ticker]).strftime('%Y-%m-%d')}-{file_end_date.strftime('%Y-%m-%d')}"
        output_file = f'coinbase_historical_price_folder/{filename}'
        df = pd.read_pickle(output_file)
        df = (df[['close','open']].rename(columns={'close': f'{ticker}_close', 'open': f'{ticker}_open'}))
        date_cond = (df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)
        df = df[date_cond]
    else:
        df = cn.save_historical_crypto_prices_from_coinbase(ticker=ticker, user_start_date=True, start_date=start_date,
                                                            end_date=end_date, save_to_file=False)
        df = (df[['close','open']].rename(columns={'close': f'{ticker}_close', 'open': f'{ticker}_open'}))
        date_cond = (df.index.get_level_values('date') >= start_date) & (df.index.get_level_values('date') <= end_date)
        df = df[date_cond]
    
    # Create Column Names
    donchian_binary_signal_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_binary_signal'
    donchian_continuous_signal_col = f'{ticker}_donchian_continuous_signal'
    donchian_continuous_signal_rank_col = f'{ticker}_donchian_continuous_signal_rank'
    trend_binary_signal_col = f'{ticker}_trend_signal'
    trend_continuous_signal_col = f'{ticker}_mavg_ribbon_slope'
    trend_continuous_signal_rank_col = f'{ticker}_mavg_ribbon_rank'
    trend_ensemble_signal_rank_col = f'{ticker}_mavg_ensemble_avg_rank'
    final_binary_signal_col = f'{ticker}_final_binary_signal'
    final_weighted_additive_signal_col = f'{ticker}_final_weighted_additive_signal'
    final_signal_col = f'{ticker}_final_signal'

    ## Generate Trend Signal in Log Space
    # df_trend = create_trend_strategy_log_space(df, ticker, mavg_start=fast_mavg, mavg_end=slow_mavg, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window)
    trend_cols = [f'{ticker}_mavg_ribbon_slope',f'{ticker}_mavg_ribbon_rank']

    trend_dfs = []
    for i in np.arange(len(fast_mavg_pair)):
        _df = create_trend_strategy_log_space(df, ticker, mavg_start=fast_mavg_pair[i], mavg_end=slow_mavg_pair[i], mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window)
        _df = _df[trend_cols]
        _df = _df.rename(columns={trend_continuous_signal_col: f'{ticker}_mavg_ribbon_slope_{fast_mavg_pair[i]}_{slow_mavg_pair[i]}',
                                  trend_continuous_signal_rank_col: f'{ticker}_mavg_ribbon_rank_{fast_mavg_pair[i]}_{slow_mavg_pair[i]}'})
        trend_dfs.append(_df)
    
    df_trend = pd.concat(trend_dfs, axis=1)
    rank_cols = [f'{ticker}_mavg_ribbon_rank_{fast_mavg_pair[0]}_{slow_mavg_pair[0]}',f'{ticker}_mavg_ribbon_rank_{fast_mavg_pair[1]}_{slow_mavg_pair[1]}']
    nan_mask = (df_trend[rank_cols].notna().all(axis=1))
    df_trend[trend_ensemble_signal_rank_col] = np.where(nan_mask, df_trend[rank_cols].mean(axis=1), np.nan)
    df_trend = pd.merge(df_trend, df[[f'{ticker}_close', f'{ticker}_open']], left_index=True, right_index=True, how='left')
    
    ## Generate Donchian Channels
    # Donchian Buy signal: Price crosses above upper band
    # Donchian Sell signal: Price crosses below lower band
    df_donchian = calculate_donchian_channel_dual_window(start_date=start_date, end_date=end_date, ticker=ticker, price_or_returns_calc=price_or_returns_calc,
                                                         entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window,
                                                         use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)

    t_1_close_col = f't_1_close'
    df_donchian[t_1_close_col] = df_donchian[f'close'].shift(1)
    donchian_entry_upper_band_col = f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_upper_band_{price_or_returns_calc}'
    donchian_entry_lower_band_col = f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_lower_band_{price_or_returns_calc}'
    donchian_entry_middle_band_col = f'{ticker}_{entry_rolling_donchian_window}_donchian_entry_middle_band_{price_or_returns_calc}'
    donchian_exit_upper_band_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_upper_band_{price_or_returns_calc}'
    donchian_exit_lower_band_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_lower_band_{price_or_returns_calc}'
    donchian_exit_middle_band_col = f'{ticker}_{exit_rolling_donchian_window}_donchian_exit_middle_band_{price_or_returns_calc}'
    shift_cols = [donchian_entry_upper_band_col, donchian_entry_lower_band_col, donchian_entry_middle_band_col,
                  donchian_exit_upper_band_col, donchian_exit_lower_band_col, donchian_exit_middle_band_col]
    for col in shift_cols:
        df_donchian[f'{col}_t_2'] = df_donchian[col].shift(1)

    # Donchian Continuous Signal
    df_donchian[donchian_continuous_signal_col] = ((df_donchian[t_1_close_col] - df_donchian[f'{donchian_entry_middle_band_col}_t_2']) /
                                                   (df_donchian[f'{donchian_entry_upper_band_col}_t_2'] - df_donchian[f'{donchian_entry_lower_band_col}_t_2']))

    ## Calculate Donchian Channel Rank
    ## Adjust the percentage ranks by 0.5 as without, the ranks go from 0 to 1. Recentering the function by giving it a steeper 
    ## slope near the origin takes into account even little information
    df_donchian[donchian_continuous_signal_rank_col] = pct_rank(df_donchian[donchian_continuous_signal_col]) - 0.5

    # Donchian Binary Signal
    gate_long_condition  = df_donchian[t_1_close_col] >= df_donchian[f'{donchian_exit_lower_band_col}_t_2']
    gate_short_condition = df_donchian[t_1_close_col] <= df_donchian[f'{donchian_exit_upper_band_col}_t_2']
    # sign of *entry* score decides direction
    entry_sign = np.sign(df_donchian[donchian_continuous_signal_col])
    # treat exact zero as "flat but allowed" (gate=1) so ranking not wiped out
    entry_sign = np.where(entry_sign == 0, 1, entry_sign)  # default to long-side keep
    df_donchian[donchian_binary_signal_col] = np.where(
        entry_sign > 0, gate_long_condition, gate_short_condition).astype(float)
    
    # Merging the Trend and Donchian Dataframes
    donchian_cols = [f'{donchian_entry_upper_band_col}_t_2', f'{donchian_entry_lower_band_col}_t_2', f'{donchian_entry_middle_band_col}_t_2',
                     f'{donchian_exit_upper_band_col}_t_2', f'{donchian_exit_lower_band_col}_t_2', f'{donchian_exit_middle_band_col}_t_2',
                     donchian_binary_signal_col, donchian_continuous_signal_col, donchian_continuous_signal_rank_col]
    df_trend = pd.merge(df_trend, df_donchian[donchian_cols], left_index=True, right_index=True, how='left')

    ## Trend and Donchian Channel Signal
    # Calculate the exponential weighted average of the ranked signals to remove short-term flip flops (whiplash)
    df_trend[[trend_ensemble_signal_rank_col, donchian_continuous_signal_rank_col]] = (
        df_trend[[trend_ensemble_signal_rank_col, donchian_continuous_signal_rank_col]].ewm(span=3, adjust=False).mean())

    # Weighted Sum of Rank Columns
    df_trend[final_weighted_additive_signal_col] = (ma_crossover_signal_weight * df_trend[trend_ensemble_signal_rank_col] +
                                                    donchian_signal_weight * df_trend[donchian_continuous_signal_rank_col])

    # Activation Scaled Signal
    if use_activation:
        final_signal_unscaled_95th_percentile = np.abs(df_trend[final_weighted_additive_signal_col]).quantile(0.95)
        if tanh_activation_constant_dict:
            k = tanh_activation_constant_dict[ticker]
            df_trend[f'{ticker}_activation'] = np.tanh(df_trend[final_weighted_additive_signal_col] * k)
        else:
            if (final_signal_unscaled_95th_percentile == 0):#| (final_signal_unscaled_95th_percentile.isnan()):
                k = 1.0
            else:
                k = np.arctanh(0.9) / final_signal_unscaled_95th_percentile
            df_trend[f'{ticker}_activation'] = np.tanh(df_trend[final_weighted_additive_signal_col] * k)
    else:
        df_trend[f'{ticker}_activation'] = df_trend[final_weighted_additive_signal_col]

    # Apply Binary Gate
    if use_donchian_exit_gate:
        df_trend[f'{ticker}_activation'] = df_trend[f'{ticker}_activation'] * df_trend[donchian_binary_signal_col]

    ## Long-Only Filter
    df_trend[final_signal_col] = np.where(long_only, np.maximum(0, df_trend[f'{ticker}_activation']), df_trend[f'{ticker}_activation'])

    return df_trend

def get_trend_ensemble_donchian_signal_for_portfolio(start_date, end_date, ticker_list, fast_mavg_pair, slow_mavg_pair, mavg_stepsize, mavg_z_score_window, entry_rolling_donchian_window, 
                                                     exit_rolling_donchian_window, use_donchian_exit_gate, donchian_signal_weight, ma_crossover_signal_weight, 
                                                     use_activation=True, tanh_activation_constant_dict=None, 
                                                     long_only=False, price_or_returns_calc='price',
                                                     use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30'):

    ## Generate trend signal for all tickers
    trend_list = []
    date_list = cn.coinbase_start_date_by_ticker_dict
    
    for ticker in ticker_list:
        # Create Column Names
        donchian_continuous_signal_col = f'{ticker}_donchian_continuous_signal'
        donchian_continuous_signal_rank_col = f'{ticker}_donchian_continuous_signal_rank'
        # trend_continuous_signal_col = f'{ticker}_mavg_ribbon_slope'
        # trend_continuous_signal_rank_col = f'{ticker}_mavg_ribbon_rank'
        trend_ensemble_signal_rank_col = f'{ticker}_mavg_ensemble_avg_rank'
        final_signal_col = f'{ticker}_final_signal'
        close_price_col = f'{ticker}_close'
        open_price_col = f'{ticker}_open'
        final_weighted_additive_signal_col = f'{ticker}_final_weighted_additive_signal'
        # lower_donchian_col = f'{ticker}_{rolling_donchian_window}_donchian_upper_band_{price_or_returns_calc}_t_2'
        # upper_donchian_col = f'{ticker}_{rolling_donchian_window}_donchian_lower_band_{price_or_returns_calc}_t_2'
        
        if pd.to_datetime(date_list[ticker]).date() > start_date:
            df_trend = generate_trend_signal_ensemble_with_donchian_channel_continuous(
                start_date=pd.to_datetime(date_list[ticker]).date(), end_date=end_date, ticker=ticker,
                fast_mavg_pair=fast_mavg_pair, slow_mavg_pair=slow_mavg_pair, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window,
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window,
                use_donchian_exit_gate=use_donchian_exit_gate, donchian_signal_weight=donchian_signal_weight, 
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, price_or_returns_calc=price_or_returns_calc, long_only=long_only,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)
        else:
            df_trend = generate_trend_signal_ensemble_with_donchian_channel_continuous(
                start_date=start_date, end_date=end_date, ticker=ticker, fast_mavg_pair=fast_mavg_pair, slow_mavg_pair=slow_mavg_pair, mavg_stepsize=mavg_stepsize,
                mavg_z_score_window=mavg_z_score_window,
                entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window,
                use_donchian_exit_gate=use_donchian_exit_gate, donchian_signal_weight=donchian_signal_weight, 
                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict, 
                ma_crossover_signal_weight=ma_crossover_signal_weight, price_or_returns_calc=price_or_returns_calc, long_only=long_only,
                use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)
            
        trend_cols = [close_price_col, open_price_col, trend_ensemble_signal_rank_col, donchian_continuous_signal_rank_col, final_weighted_additive_signal_col, final_signal_col]
        # trend_cols = [close_price_col, open_price_col, lower_donchian_col, upper_donchian_col, donchian_continuous_signal_col, donchian_continuous_signal_rank_col,
        #               trend_continuous_signal_col, trend_continuous_signal_rank_col, final_weighted_additive_signal_col, final_signal_col]
        df_trend = df_trend[trend_cols]
        trend_list.append(df_trend)

    df_trend = pd.concat(trend_list, axis=1)

    return df_trend

In [578]:
def apply_target_volatility_position_sizing_continuous_strategy_ensemble(start_date, end_date, ticker_list, fast_mavg_pair, slow_mavg_pair, mavg_stepsize, mavg_z_score_window, ma_crossover_signal_weight,
                                                                         donchian_signal_weight, entry_rolling_donchian_window, exit_rolling_donchian_window, use_donchian_exit_gate, 
                                                                         use_activation=True, tanh_activation_constant_dict=None, long_only=False,
                                                                         initial_capital=15000, rolling_cov_window=20, volatility_window=20,
                                                                         rolling_atr_window=20, atr_multiplier=0.5,
                                                                         transaction_cost_est=0.001, passive_trade_rate=0.05,
                                                                         use_coinbase_data=True, use_saved_files=True, saved_file_end_date='2025-06-30', 
                                                                         rolling_sharpe_window=50, cash_buffer_percentage=0.10, annualized_target_volatility=0.20,
                                                                         annual_trading_days=365, use_specific_start_date=False,
                                                                         signal_start_date=None):

    ## Check if data is available for all the tickers
    date_list = cn.coinbase_start_date_by_ticker_dict
    ticker_list = [ticker for ticker in ticker_list if pd.Timestamp(date_list[ticker]).date() < end_date]
    
    print('Generating Moving Average Ribbon Signal!!')
    ## Generate Trend Signal for all tickers
    df_trend = get_trend_ensemble_donchian_signal_for_portfolio(start_date=start_date, end_date=end_date, ticker_list=ticker_list, fast_mavg_pair=fast_mavg_pair,
                                                                slow_mavg_pair=slow_mavg_pair, mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, 
                                                                entry_rolling_donchian_window=entry_rolling_donchian_window, 
                                                                exit_rolling_donchian_window=exit_rolling_donchian_window, use_donchian_exit_gate=use_donchian_exit_gate,
                                                                donchian_signal_weight=donchian_signal_weight, ma_crossover_signal_weight=ma_crossover_signal_weight, 
                                                                use_activation=use_activation, tanh_activation_constant_dict=tanh_activation_constant_dict,
                                                                long_only=long_only, use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files, saved_file_end_date=saved_file_end_date)

    print('Generating Volatility Adjusted Trend Signal!!')
    ## Get Volatility Adjusted Trend Signal
    df_signal = size_cont.get_volatility_adjusted_trend_signal_continuous(df_trend, ticker_list, volatility_window, annual_trading_days)

    print('Getting Average True Range for Stop Loss Calculation!!')
    ## Get Average True Range for Stop Loss Calculation
    df_atr = size_cont.get_average_true_range_portfolio(start_date=start_date, end_date=end_date, ticker_list=ticker_list, rolling_atr_window=rolling_atr_window,
                                                        price_or_returns_calc='price', use_coinbase_data=use_coinbase_data, use_saved_files=use_saved_files,
                                                        saved_file_end_date=saved_file_end_date)
    df_signal = pd.merge(df_signal, df_atr, left_index=True, right_index=True, how='left')

    print('Calculating Volatility Targeted Position Size and Cash Management!!')
    ## Get Target Volatility Position Sizing and Run Cash Management
    df = size_cont.get_target_volatility_daily_portfolio_positions(df_signal, ticker_list=ticker_list, initial_capital=initial_capital, rolling_cov_window=rolling_cov_window,
                                                                   rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier, cash_buffer_percentage=cash_buffer_percentage,
                                                                   annualized_target_volatility=annualized_target_volatility, transaction_cost_est=transaction_cost_est,
                                                                   passive_trade_rate=passive_trade_rate, notional_threshold_pct=notional_threshold_pct,
                                                                   cooldown_counter_threshold=cooldown_counter_threshold, annual_trading_days=annual_trading_days,
                                                                   use_specific_start_date=use_specific_start_date, signal_start_date=signal_start_date)

    print('Calculating Portfolio Performance!!')
    ## Calculate Portfolio Performance
    df = size_bin.calculate_portfolio_returns(df, rolling_sharpe_window)

    return df

In [610]:
def run_walk_forward_moving_avg_ribbon_locked_pairs_ensemble(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'fast_mavg_pair', 'slow_mavg_pair', 'mavg_stepsize', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    fast_mavg_pair = [10, 20]
    slow_mavg_pair = [40, 200]
    mavg_stepsize = 8
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        # print(f'In Sample Start: {start_date_is}, In Sample End: {end_date_is}, Out of Sample Start: {start_date_os}, Out of Sample End: {end_date_os}')
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        
        ## In Sample Dataframe
        print('Pulling In Sample Data!!')
        df_is = apply_target_volatility_position_sizing_continuous_strategy_ensemble(
            start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg_pair=fast_mavg_pair, slow_mavg_pair=slow_mavg_pair,
            mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
            use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
            initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
            rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
            transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
            rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
            annualized_target_volatility=annualized_target_volatility,
            annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_is)
        df_is = df_is[df_is.index >= start_date_is]
        
        print('Calculating In Sample Asset Returns!!')
        df_is = calculate_asset_level_returns(df_is, end_date, ticker_list)

        ## In Sample Performance Metrics
        print('Getting In Sample Performance Metrics!!')
        row_parameters_is = {
            'sampling_category': 'in_sample',
            'start_date': start_date_is,
            'end_date': end_date_is,
            'fast_mavg_pair': fast_mavg_pair,
            'slow_mavg_pair': slow_mavg_pair,
            'mavg_stepsize': mavg_stepsize
        }
        portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                           strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                           passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

        print('Getting In Sample Asset Performance!!')
        for ticker in ticker_list:
            ## In Sample
            ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                 strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                 annual_trading_days=365, include_transaction_costs_and_fees=False)
            ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
            ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
            portfolio_perf_metrics_is.update(ticker_perf_metrics_is)

        row_parameters_is.update(portfolio_perf_metrics_is)

        ## Assign in sample and out of sample metrics to performance dataframe
        df_performance.loc[df_performance.shape[0]] = row_parameters_is

        ## Out of Sample Dataframe
        df_os = apply_target_volatility_position_sizing_continuous_strategy_ensemble(
            start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg_pair=fast_mavg_pair, slow_mavg_pair=slow_mavg_pair,
            mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
            use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
            initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
            rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
            transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
            rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
            annualized_target_volatility=annualized_target_volatility,
            annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_os)

        df_os = df_os[df_os.index >= start_date_os]
        print('Calculating Out of Sample Asset Returns!!')
        df_os = calculate_asset_level_returns(df_os, end_date, ticker_list)

        ## Out of Sample Performance Metrics
        print('Pulling Out of Sample Performance Metrics!!')
        row_parameters_os = {
            'sampling_category': 'out_sample',
            'start_date': start_date_os,
            'end_date': end_date_os,
            'fast_mavg_pair': fast_mavg_pair,
            'slow_mavg_pair': slow_mavg_pair,
            'mavg_stepsize': mavg_stepsize
        }
        portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                           strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                           passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

        print('Getting Out of Sample Asset Performance!!')
        for ticker in ticker_list:
            ## Out of Sample
            ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                 strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                 annual_trading_days=365, include_transaction_costs_and_fees=False)
            ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
            ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
            portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
        
        row_parameters_os.update(portfolio_perf_metrics_os)

        ## Assign in sample and out of sample metrics to performance dataframe
        df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()

    return df_performance

In [612]:
%%time
df_performance_locked_ensemble = run_walk_forward_moving_avg_ribbon_locked_pairs_ensemble(start_date='2022-04-01', end_date='2025-10-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating Out of Sample Asset Returns!!
Pulling Out of Sample Per

In [640]:
df_performance_locked_ensemble

,sampling_category,start_date,end_date,fast_mavg_pair,slow_mavg_pair,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,"[10, 20]","[40, 200]",8,-0.049185,0.039248,-0.095947,0.496530,-0.512630,468 days,0.472628,0.170814,0.864433,1746.0,-0.005784,0.068358,-0.025076,-0.031857,0.154608,-0.360689,0.204368,-0.537511,-0.554011,0.777073,0.130840,0.121312,0.124615,0.132980,0.134523,-0.181198,-0.093109,-0.177142,-0.128111,-0.178202
1,out_sample,2023-10-01,2024-03-31,"[10, 20]","[40, 200]",8,1.610961,2.091635,6.435787,0.504194,-0.250313,96 days,0.524590,1.550968,0.122647,609.0,0.331618,0.226558,0.418587,0.215327,0.189715,1.583347,1.048775,2.287883,1.235677,1.168549,0.161027,0.162177,0.135925,0.124928,0.114486,-0.075783,-0.073005,-0.067603,-0.079684,-0.074913
2,in_sample,2022-10-01,2024-03-31,"[10, 20]","[40, 200]",8,0.413159,0.892274,1.642796,0.451094,-0.251497,261 days,0.498175,1.228382,0.219832,1750.0,0.144140,0.101575,0.111600,0.109234,0.095309,0.708961,0.446203,0.518426,0.517991,0.439296,0.135658,0.126372,0.125788,0.121198,0.111220,-0.076969,-0.075559,-0.115006,-0.082471,-0.098658
3,out_sample,2024-04-01,2024-09-30,"[10, 20]","[40, 200]",8,-0.238490,-0.405094,-0.750623,0.499442,-0.317723,84 days,0.508197,-0.216787,0.828617,554.0,-0.164543,-0.074534,-0.074914,0.069158,0.113001,-1.958030,-0.741064,-0.905398,0.195917,0.509594,0.114688,0.155962,0.130892,0.146528,0.132554,-0.103703,-0.104797,-0.091129,-0.115163,-0.074384
4,in_sample,2023-04-01,2024-09-30,"[10, 20]","[40, 200]",8,0.353475,0.786541,1.111585,0.459879,-0.317992,143 days,0.491803,1.096680,0.273263,1709.0,0.014105,0.053269,0.117895,0.123619,0.174589,-0.220805,0.092384,0.556925,0.616071,1.013359,0.124753,0.142149,0.127034,0.122464,0.118867,-0.106345,-0.149676,-0.093699,-0.116582,-0.077114
5,out_sample,2024-10-01,2025-03-31,"[10, 20]","[40, 200]",8,0.274055,0.632967,0.631459,0.525098,-0.434003,112 days,0.456044,0.514226,0.607721,582.0,0.232269,0.024915,-0.057879,-0.022229,0.200078,1.212703,-0.113370,-0.659251,-0.277164,0.953692,0.141600,0.135739,0.150860,0.192344,0.155116,-0.092162,-0.109086,-0.138276,-0.161326,-0.100056
6,in_sample,2023-10-01,2025-03-31,"[10, 20]","[40, 200]",8,0.329983,0.720890,0.724816,0.514796,-0.455265,266 days,0.490876,1.001867,0.316851,1730.0,0.095608,0.057966,0.064381,0.071663,0.154669,0.374761,0.125564,0.167474,0.209296,0.786829,0.140899,0.152599,0.138736,0.157985,0.133720,-0.117349,-0.178444,-0.151024,-0.163057,-0.099419
7,out_sample,2025-04-01,2025-07-31,"[10, 20]","[40, 200]",8,-0.166278,-0.194042,-0.814960,0.530898,-0.204032,82 days,0.428571,-0.049608,0.960545,289.0,0.065361,0.168789,-0.042778,-0.008373,-0.106841,0.184771,0.698459,-0.762995,-0.329155,-1.550370,0.113169,0.176233,0.113621,0.143788,0.101645,-0.039081,-0.059483,-0.054763,-0.075621,-0.046867


In [654]:
df_performance_locked_ensemble.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Locked_Pairs_Ensemble-2022-04-01-2025-10-01.pickle')

In [624]:
df_performance_locked_ensemble['fast_mavg_pair'] = df_performance_locked_ensemble['fast_mavg_pair'].astype(str)
df_performance_locked_ensemble['slow_mavg_pair'] = df_performance_locked_ensemble['slow_mavg_pair'].astype(str)

In [646]:
df_performance_locked_ensemble_is = df_performance_locked_ensemble[(df_performance_locked_ensemble.sampling_category == 'in_sample')].reset_index(drop=True)
df_performance_locked_ensemble_os = df_performance_locked_ensemble[(df_performance_locked_ensemble.sampling_category == 'out_sample')].reset_index(drop=True)

In [650]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_locked_ensemble_is.groupby(['fast_mavg_pair','slow_mavg_pair']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                               median      mean       std   
fast_mavg_pair slow_mavg_pair                                               
[10, 20]       [40, 200]                     0.753715  0.609738  0.386825   

                              annualized_return                      \
                                         median      mean       std   
fast_mavg_pair slow_mavg_pair                                         
[10, 20]       [40, 200]               0.341729  0.261858  0.210297   

                              max_drawdown                      \
                                    median      mean       std   
fast_mavg_pair slow_mavg_pair                                    
[10, 20]       [40, 200]         -0.386628 -0.384346  0.120468   

                              BTC-USD_annualized_sharpe_ratio            \
                                                       median      mean   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]                             0.076978  0.125557   

                                        ETH-USD_annualized_sharpe_ratio  \
                                    std                          median   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]       0.502946                        0.164966   

                                                  \
                                  mean       std   
fast_mavg_pair slow_mavg_pair                      
[10, 20]       [40, 200]       0.21713  0.159774   

                              SOL-USD_annualized_sharpe_ratio            \
                                                       median      mean   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]                              0.34295  0.176328   

                                        ADA-USD_annualized_sharpe_ratio  \
                                    std                          median   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]       0.507126                        0.363643   

                                                   \
                                   mean       std   
fast_mavg_pair slow_mavg_pair                       
[10, 20]       [40, 200]       0.197336  0.530039   

                              AVAX-USD_annualized_sharpe_ratio            \
                                                        median      mean   
fast_mavg_pair slow_mavg_pair                                              
[10, 20]       [40, 200]                              0.781951  0.754139   

                                         
                                    std  
fast_mavg_pair slow_mavg_pair            
[10, 20]       [40, 200]       0.236584

In [651]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_locked_ensemble_os.groupby(['fast_mavg_pair','slow_mavg_pair']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                               median      mean       std   
fast_mavg_pair slow_mavg_pair                                               
[10, 20]       [40, 200]                     0.219463  0.531367  1.132539   

                              annualized_return                      \
                                         median      mean       std   
fast_mavg_pair slow_mavg_pair                                         
[10, 20]       [40, 200]               0.053888  0.370062  0.857719   

                              max_drawdown                    \
                                    median      mean     std   
fast_mavg_pair slow_mavg_pair                                  
[10, 20]       [40, 200]         -0.284018 -0.301518  0.0999   

                              BTC-USD_annualized_sharpe_ratio            \
                                                       median      mean   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]                             0.698737  0.255698   

                                        ETH-USD_annualized_sharpe_ratio  \
                                    std                          median   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]       1.589983                        0.292545   

                                                 \
                                 mean       std   
fast_mavg_pair slow_mavg_pair                     
[10, 20]       [40, 200]       0.2232  0.806335   

                              SOL-USD_annualized_sharpe_ratio           \
                                                       median     mean   
fast_mavg_pair slow_mavg_pair                                            
[10, 20]       [40, 200]                            -0.711123 -0.00994   

                                        ADA-USD_annualized_sharpe_ratio  \
                                    std                          median   
fast_mavg_pair slow_mavg_pair                                             
[10, 20]       [40, 200]       1.535202                       -0.040623   

                                                   \
                                   mean       std   
fast_mavg_pair slow_mavg_pair                       
[10, 20]       [40, 200]       0.206319  0.725758   

                              AVAX-USD_annualized_sharpe_ratio            \
                                                        median      mean   
fast_mavg_pair slow_mavg_pair                                              
[10, 20]       [40, 200]                              0.731643  0.270366   

                                         
                                    std  
fast_mavg_pair slow_mavg_pair            
[10, 20]       [40, 200]       1.244451

In [658]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_locked_is.groupby(['fast_mavg','slow_mavg']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                     median      mean       std   
fast_mavg slow_mavg                                               
10        40                       0.469941  0.486367  0.307246   
          80                       0.648455  0.534802  0.333616   
          100                      0.672112  0.530125  0.348356   
          120                      0.696390  0.535961  0.372571   
20        80                       0.449599  0.346947  0.418220   
          160                      1.086596  0.839609  0.605151   
          200                      1.204199  0.916628  0.685358   
          240                      1.218594  0.915583  0.725632   

                    annualized_return                     max_drawdown  \
                               median      mean       std       median   
fast_mavg slow_mavg                                                      
10        40                 0.182220  0.188671  0.169176    -0.517406   
          80                 0.268516  0.216542  0.184212    -0.393469   
          100                0.281948  0.214270  0.189424    -0.384649   
          120                0.296422  0.219225  0.201821    -0.380639   
20        80                 0.161778  0.121238  0.214847    -0.377228   
          160                0.550028  0.432450  0.367043    -0.294452   
          200                0.636918  0.496599  0.421799    -0.294313   
          240                0.648051  0.501537  0.442926    -0.310095   

                                        BTC-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40        -0.472593  0.107770                        0.170938   
          80        -0.391667  0.103504                        0.178800   
          100       -0.384493  0.104503                        0.190373   
          120       -0.391264  0.111139                        0.158404   
20        80        -0.394192  0.096988                        0.050071   
          160       -0.347066  0.157627                        0.148320   
          200       -0.349551  0.162287                        0.203160   
          240       -0.357454  0.158796                        0.236793   

                                        ETH-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         0.253981  0.487392                        0.112351   
          80         0.176116  0.491512                        0.212634   
          100        0.164856  0.499514                        0.339929   
          120        0.144164  0.515491                        0.382036   
20        80         0.051078  0.464762                        0.200669   
          160        0.226369  0.574672                        0.423255   
          200        0.292045  0.597792                        0.363187   
          240        0.315811  0.583662                        0.285261   

                                        SOL-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         0.092238  0.311474                        0.191874   
          80         0.193696  0.131837                        0.099853   
          100        0.302620  0.190747                        0.011484   
          120        0.310084  0.246758                        0.003039   
20        80         0.158447  0.309272                       -0.255463   
          160        0.428898  0.370375                        0.192056   
          200        0.386421  0.375725                        0.378501   
          240        0.331049  0.388349                        0.

In [656]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_locked_os.groupby(['fast_mavg','slow_mavg']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                     median      mean       std   
fast_mavg slow_mavg                                               
10        40                       1.254068  1.254068       NaN   
20        200                      1.179849  1.179849       NaN   
          240                     -0.873101 -0.873101  1.293271   

                    annualized_return                     max_drawdown  \
                               median      mean       std       median   
fast_mavg slow_mavg                                                      
10        40                 0.726777  0.726777       NaN    -0.349545   
20        200                0.679147  0.679147       NaN    -0.303374   
          240               -0.306935 -0.306935  0.362915    -0.258926   

                                        BTC-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40        -0.349545       NaN                        1.334478   
20        200       -0.303374       NaN                        1.120584   
          240       -0.258926  0.073424                       -1.065919   

                                        ETH-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         1.334478       NaN                       -0.356197   
20        200        1.120584       NaN                        0.192144   
          240       -1.065919  0.336777                       -0.861801   

                                       SOL-USD_annualized_sharpe_ratio  \
                         mean      std                          median   
fast_mavg slow_mavg                                                      
10        40        -0.356197      NaN                        1.702805   
20        200        0.192144      NaN                       -1.271911   
          240       -0.861801  0.07901                       -1.246547   

                                        ADA-USD_annualized_sharpe_ratio  \
                         mean       std                          median   
fast_mavg slow_mavg                                                       
10        40         1.702805       NaN                        0.861648   
20        200       -1.271911       NaN                        0.840731   
          240       -1.246547  0.435426                        0.013891   

                                       AVAX-USD_annualized_sharpe_ratio  \
                         mean      std                           median   
fast_mavg slow_mavg                                                       
10        40         0.861648      NaN                         0.586691   
20        200        0.840731      NaN                         1.441356   
          240        0.013891  1.30374                        -2.304297   

                                         
                         mean       std  
fast_mavg slow_mavg                      
10        40         0.586691       NaN  
20        200        1.441356       NaN  
          240       -2.304297  3.735051

In [556]:
fast_mavg_pair = [10, 20]
slow_mavg_pair = [40, 200]
df_ensemble = apply_target_volatility_position_sizing_continuous_strategy_ensemble(
    start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg_pair=fast_mavg_pair, slow_mavg_pair=slow_mavg_pair,
    mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
    use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
    initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
    rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
    transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
    rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
    annualized_target_volatility=annualized_target_volatility,
    annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_is)

Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!


In [558]:
calculate_risk_and_performance_metrics(df_ensemble, strategy_daily_return_col=f'portfolio_daily_pct_returns', strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                       passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

{'annualized_return': -0.09409660441736178,
 'annualized_sharpe_ratio': -0.17716316472428378,
 'calmar_ratio': -0.20538688962320228,
 'annualized_std_dev': 0.5176764813826676,
 'max_drawdown': -0.4581431881557245,
 'max_drawdown_duration': Timedelta('357 days 00:00:00'),
 'hit_rate': 0.28100263852242746,
 't_statistic': -0.07698846208713656,
 'p_value': 0.9386530667478855,
 'trade_count': 1423.0}

In [560]:
fast_mavg_pair = [10, 20]
slow_mavg_pair = [40, 200]
df_is = apply_target_volatility_position_sizing_continuous_strategy(
    start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
    mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
    use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
    initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
    rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
    transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
    rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
    annualized_target_volatility=annualized_target_volatility,
    annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_is)

Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!


In [561]:
calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns', strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                       passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

{'annualized_return': 0.007977830442082556,
 'annualized_sharpe_ratio': 0.09553987165826007,
 'calmar_ratio': 0.01829284086704859,
 'annualized_std_dev': 0.5224810511793386,
 'max_drawdown': -0.4361176320323896,
 'max_drawdown_duration': Timedelta('304 days 00:00:00'),
 'hit_rate': 0.2757255936675462,
 't_statistic': 0.31470411519315744,
 'p_value': 0.7530730387930402,
 'trade_count': 1412.0}

## Run Walk Forward Analysis for Moving Average Crossover Stepsize

In [672]:
import itertools

def generate_moving_avg_stepsize_params():
    parameter_grid = {
        "fast_mavg": [20],
        "slow_mavg": [200],
        "stepsize":[2, 4, 6, 8, 10, 12, 14],
    }
    keys, values = zip(*parameter_grid.items())
    for prod in itertools.product(*values):
        yield dict(zip(keys, prod))

In [678]:
def run_walk_forward_moving_avg_ribbon_stepsize(start_date, end_date, ticker_list):

    start_date = pd.Timestamp(start_date).date()
    end_date = pd.Timestamp(end_date).date()
    perf_cols = ['sampling_category', 'start_date', 'end_date', 'fast_mavg', 'slow_mavg', 'mavg_stepsize', 'annualized_return', 'annualized_sharpe_ratio', 'calmar_ratio',
                 'annualized_std_dev', 'max_drawdown', 'max_drawdown_duration', 'hit_rate', 't_statistic', 'p_value', 'trade_count']
    ticker_perf_cols = ['annualized_return','annualized_sharpe_ratio','annualized_std_dev','max_drawdown']
    perf_cols.extend([f'{ticker}_{col}' for col in ticker_perf_cols for ticker in ticker_list])
    
    df_performance = pd.DataFrame(columns=perf_cols)
    
    IS_LEN = pd.DateOffset(months=18)
    OS_LEN = pd.DateOffset(months=6)
    start_date_is = start_date
    last_available_date = pd.Timestamp('2025-07-31').date()
    WARMUP_DAYS = 323
    while True:
        end_date_is = (start_date_is + IS_LEN - pd.Timedelta(days=1)).date()
        start_date_os = (end_date_is + pd.Timedelta(days=1))
        end_date_os = (start_date_os + OS_LEN - pd.Timedelta(days=1)).date()
        fmt = "%Y-%m-%d"
        
        fields = [
            ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
            ("IS start",          start_date_is),
            ("IS end",            end_date_is),
            ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
            ("OS start",          start_date_os),
            ("OS end",            end_date_os),
        ]
        
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        # print(f'In Sample Start: {start_date_is}, In Sample End: {end_date_is}, Out of Sample Start: {start_date_os}, Out of Sample End: {end_date_os}')
        if end_date_os > end_date - pd.Timedelta(days=1):
            break

        if end_date_os > last_available_date:
            print('end_date_os > last_available_date')
            end_date_os = last_available_date
            fields = [
                ("Warm-up IS start",  start_date_is - pd.Timedelta(days=WARMUP_DAYS)),
                ("IS start",          start_date_is),
                ("IS end",            end_date_is),
                ("Warm-up OS start",  start_date_os - pd.Timedelta(days=WARMUP_DAYS)),
                ("OS start",          start_date_os),
                ("OS end",            end_date_os),
            ]
        
        print("Run Dates: ")
        print(", ".join(f"{k}: {v:{fmt}}" for k, v in fields))
        for params in generate_moving_avg_stepsize_params():
            print(params)
            fast_mavg = params['fast_mavg']
            slow_mavg = params['slow_mavg']
            mavg_stepsize = params['stepsize']
            print(fast_mavg, slow_mavg, mavg_stepsize)
            
            ## In Sample Dataframe
            print('Pulling In Sample Data!!')
            df_is = apply_target_volatility_position_sizing_continuous_strategy(
                start_date=start_date_is - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_is, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
                mavg_stepsize=mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
                use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
                initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
                rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
                transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
                rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
                annualized_target_volatility=annualized_target_volatility,
                annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_is)
            df_is = df_is[df_is.index >= start_date_is]
            
            print('Calculating In Sample Asset Returns!!')
            df_is = calculate_asset_level_returns(df_is, end_date, ticker_list)

            ## In Sample Performance Metrics
            print('Getting In Sample Performance Metrics!!')
            row_parameters_is = {
                'sampling_category': 'in_sample',
                'start_date': start_date_is,
                'end_date': end_date_is,
                'fast_mavg': fast_mavg,
                'slow_mavg': slow_mavg,
                'mavg_stepsize': mavg_stepsize
            }
            portfolio_perf_metrics_is = calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                               strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                               passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

            print('Getting In Sample Asset Performance!!')
            for ticker in ticker_list:
                ## In Sample
                ticker_perf_metrics_is = perf.calculate_risk_and_performance_metrics(df_is, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                     strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                     annual_trading_days=365, include_transaction_costs_and_fees=False)
                ticker_perf_metrics_is = {key: ticker_perf_metrics_is[key] for key in ticker_perf_cols}
                ticker_perf_metrics_is = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_is.items()}
                portfolio_perf_metrics_is.update(ticker_perf_metrics_is)

            row_parameters_is.update(portfolio_perf_metrics_is)

            ## Assign in sample and out of sample metrics to performance dataframe
            df_performance.loc[df_performance.shape[0]] = row_parameters_is

        ## Get Moving Average and Donchian Channel Weights with best performing in-sample Sharpe Ratio
        in_sample_cond = (df_performance['sampling_category'] == 'in_sample')
        date_cond = (df_performance['start_date'] == start_date_is)# & (df_performance['end_date'] == end_date_is)
        best_in_sample_mavg_stepsize = df_performance[in_sample_cond & date_cond].sort_values('annualized_sharpe_ratio', ascending=False)['mavg_stepsize'].iloc[0]
        print(f'Best In Sample Mavg Stepsize: {best_in_sample_mavg_stepsize}')

        ## Out of Sample Dataframe
        df_os = apply_target_volatility_position_sizing_continuous_strategy(
            start_date=start_date_os - pd.Timedelta(days=WARMUP_DAYS), end_date=end_date_os, ticker_list=ticker_list, fast_mavg=fast_mavg, slow_mavg=slow_mavg,
            mavg_stepsize=best_in_sample_mavg_stepsize, mavg_z_score_window=mavg_z_score_window, entry_rolling_donchian_window=entry_rolling_donchian_window, exit_rolling_donchian_window=exit_rolling_donchian_window, 
            use_donchian_exit_gate=use_donchian_exit_gate, ma_crossover_signal_weight=ma_crossover_signal_weight, donchian_signal_weight=donchian_signal_weight, 
            initial_capital=initial_capital, rolling_cov_window=rolling_cov_window, volatility_window=volatility_window,
            rolling_atr_window=rolling_atr_window, atr_multiplier=atr_multiplier,
            transaction_cost_est=transaction_cost_est, passive_trade_rate=passive_trade_rate, use_coinbase_data=use_coinbase_data,
            rolling_sharpe_window=rolling_sharpe_window, cash_buffer_percentage=cash_buffer_percentage,
            annualized_target_volatility=annualized_target_volatility,
            annual_trading_days=annual_trading_days, use_specific_start_date=True, signal_start_date=start_date_os)

        df_os = df_os[df_os.index >= start_date_os]
        print('Calculating Out of Sample Asset Returns!!')
        df_os = calculate_asset_level_returns(df_os, end_date, ticker_list)

        ## Out of Sample Performance Metrics
        print('Pulling Out of Sample Performance Metrics!!')
        row_parameters_os = {
            'sampling_category': 'out_sample',
            'start_date': start_date_os,
            'end_date': end_date_os,
            'fast_mavg': fast_mavg,
            'slow_mavg': slow_mavg,
            'mavg_stepsize': best_in_sample_mavg_stepsize
        }
        portfolio_perf_metrics_os = calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'portfolio_daily_pct_returns',
                                                                           strategy_trade_count_col=f'count_of_positions', include_transaction_costs_and_fees=False,
                                                                           passive_trade_rate=0.05, annual_trading_days=365, transaction_cost_est=0.001)

        print('Getting Out of Sample Asset Performance!!')
        for ticker in ticker_list:
            ## Out of Sample
            ticker_perf_metrics_os = perf.calculate_risk_and_performance_metrics(df_os, strategy_daily_return_col=f'{ticker}_daily_pct_returns',
                                                                                 strategy_trade_count_col=f'{ticker}_position_count', 
                                                                                 annual_trading_days=365, include_transaction_costs_and_fees=False)
            ticker_perf_metrics_os = {key: ticker_perf_metrics_os[key] for key in ticker_perf_cols}
            ticker_perf_metrics_os = {f'{ticker}_{key}': value for key, value in ticker_perf_metrics_os.items()}
            portfolio_perf_metrics_os.update(ticker_perf_metrics_os)
        
        row_parameters_os.update(portfolio_perf_metrics_os)

        ## Assign in sample and out of sample metrics to performance dataframe
        df_performance.loc[df_performance.shape[0]] = row_parameters_os

        start_date_is = (start_date_is + OS_LEN).date()

    return df_performance

In [680]:
%%time
df_performance_stepsize_1 = run_walk_forward_moving_avg_ribbon_stepsize(start_date='2022-04-01', end_date='2024-04-01', ticker_list=ticker_list)

Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
Run Dates: 
Warm-up IS start: 2021-05-13, IS start: 2022-04-01, IS end: 2023-09-30, Warm-up OS start: 2022-11-12, OS start: 2023-10-01, OS end: 2024-03-31
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 2}
20 200 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 4}
20 200 4
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Tar

In [682]:
df_performance_stepsize_1

,sampling_category,start_date,end_date,fast_mavg,slow_mavg,mavg_stepsize,annualized_return,annualized_sharpe_ratio,calmar_ratio,annualized_std_dev,max_drawdown,max_drawdown_duration,hit_rate,t_statistic,p_value,trade_count,BTC-USD_annualized_return,ETH-USD_annualized_return,SOL-USD_annualized_return,ADA-USD_annualized_return,AVAX-USD_annualized_return,BTC-USD_annualized_sharpe_ratio,ETH-USD_annualized_sharpe_ratio,SOL-USD_annualized_sharpe_ratio,ADA-USD_annualized_sharpe_ratio,AVAX-USD_annualized_sharpe_ratio,BTC-USD_annualized_std_dev,ETH-USD_annualized_std_dev,SOL-USD_annualized_std_dev,ADA-USD_annualized_std_dev,AVAX-USD_annualized_std_dev,BTC-USD_max_drawdown,ETH-USD_max_drawdown,SOL-USD_max_drawdown,ADA-USD_max_drawdown,AVAX-USD_max_drawdown
0,in_sample,2022-04-01,2023-09-30,20,200,2,-0.150370,-0.179808,-0.252897,0.501393,-0.594588,468 days,0.470803,-0.099719,0.920604,1742.0,0.009476,0.040822,-0.053954,-0.094587,0.122322,-0.233496,-0.017475,-0.758231,-1.016960,0.570276,0.133534,0.118001,0.128078,0.138552,0.132854,-0.173505,-0.132036,-0.222580,-0.177763,-0.184340
1,in_sample,2022-04-01,2023-09-30,20,200,4,-0.126444,-0.121251,-0.214068,0.503915,-0.590673,468 days,0.478102,-0.028464,0.977303,1730.0,0.011189,0.036889,-0.043694,-0.096005,0.132405,-0.222952,-0.049585,-0.670731,-1.029087,0.626274,0.132651,0.118321,0.128126,0.138473,0.136010,-0.173357,-0.132052,-0.214291,-0.177852,-0.184488
2,in_sample,2022-04-01,2023-09-30,20,200,6,-0.120865,-0.106578,-0.206647,0.505195,-0.584885,468 days,0.478102,-0.010847,0.991349,1711.0,0.007716,0.037903,-0.042878,-0.093344,0.137337,-0.250807,-0.041006,-0.661833,-1.010071,0.647444,0.132119,0.118467,0.128481,0.138170,0.138921,-0.173635,-0.128750,-0.212475,-0.177479,-0.184635
3,in_sample,2022-04-01,2023-09-30,20,200,8,-0.120016,-0.104408,-0.205883,0.504893,-0.582934,468 days,0.478102,-0.008229,0.993437,1713.0,0.006743,0.038041,-0.041140,-0.092815,0.136619,-0.259248,-0.039606,-0.647258,-1.007143,0.640101,0.131772,0.118661,0.128542,0.138006,0.139691,-0.173823,-0.127845,-0.210354,-0.176903,-0.185172
4,in_sample,2022-04-01,2023-09-30,20,200,10,-0.121292,-0.107298,-0.208537,0.505352,-0.581632,468 days,0.472628,-0.011768,0.990615,1710.0,0.006388,0.037127,-0.041848,-0.091374,0.136297,-0.262440,-0.046934,-0.654293,-0.995232,0.635940,0.131612,0.118813,0.128336,0.138035,0.140288,-0.174075,-0.127268,-0.209702,-0.174921,-0.185085
5,in_sample,2022-04-01,2023-09-30,20,200,12,-0.120905,-0.106424,-0.207955,0.505606,-0.581401,468 days,0.474453,-0.010696,0.991470,1698.0,0.005259,0.036287,-0.040603,-0.091600,0.135414,-0.271270,-0.053705,-0.646606,-0.996847,0.628301,0.131555,0.118918,0.127922,0.138060,0.140878,-0.174366,-0.127972,-0.207585,-0.174912,-0.184947
6,in_sample,2022-04-01,2023-09-30,20,200,14,-0.120944,-0.106446,-0.208177,0.506075,-0.580967,468 days,0.472628,-0.010735,0.991438,1693.0,0.005195,0.036509,-0.041181,-0.092125,0.135686,-0.271986,-0.051844,-0.651356,-1.000983,0.628639,0.131483,0.118944,0.127920,0.138073,0.141272,-0.173857,-0.127284,-0.207828,-0.175694,-0.184864
7,out_sample,2023-10-01,2024-03-31,20,200,8,2.597297,2.696280,12.335271,0.509171,-0.210559,69 days,0.540984,1.977677,0.049475,582.0,0.242743,0.288749,0.511166,0.348590,0.356432,1.198381,1.334427,2.928442,2.070582,2.187350,0.152941,0.165363,0.127481,0.124974,0.122746,-0.090062,-0.075006,-0.040871,-0.059812,-0.051722


In [684]:
df_performance_stepsize_1.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Stepsize-2022-04-01-2024-04-01.pickle')

In [686]:
%%time
df_performance_stepsize_2 = run_walk_forward_moving_avg_ribbon_stepsize(start_date='2022-10-01', end_date='2024-10-01', ticker_list=ticker_list)

Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
Run Dates: 
Warm-up IS start: 2021-11-12, IS start: 2022-10-01, IS end: 2024-03-31, Warm-up OS start: 2023-05-14, OS start: 2024-04-01, OS end: 2024-09-30
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 2}
20 200 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 4}
20 200 4
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Tar

In [687]:
df_performance_stepsize_2.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Stepsize-2022-10-01-2024-10-01.pickle')

In [688]:
%%time
df_performance_stepsize_3 = run_walk_forward_moving_avg_ribbon_stepsize(start_date='2023-04-01', end_date='2025-04-01', ticker_list=ticker_list)

Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
Run Dates: 
Warm-up IS start: 2022-05-13, IS start: 2023-04-01, IS end: 2024-09-30, Warm-up OS start: 2023-11-13, OS start: 2024-10-01, OS end: 2025-03-31
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 2}
20 200 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 4}
20 200 4
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Tar

In [689]:
df_performance_stepsize_3.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Stepsize-2023-04-01-2025-04-01.pickle')

In [690]:
%%time
df_performance_stepsize_4 = run_walk_forward_moving_avg_ribbon_stepsize(start_date='2023-10-01', end_date='2025-10-01', ticker_list=ticker_list)

Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-09-30
end_date_os > last_available_date
Run Dates: 
Warm-up IS start: 2022-11-12, IS start: 2023-10-01, IS end: 2025-03-31, Warm-up OS start: 2024-05-13, OS start: 2025-04-01, OS end: 2025-07-31
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 2}
20 200 2
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calculation!!
Calculating Volatility Targeted Position Size and Cash Management!!
Calculating Portfolio Performance!!
Calculating In Sample Asset Returns!!
Getting In Sample Performance Metrics!!
Getting In Sample Asset Performance!!
{'fast_mavg': 20, 'slow_mavg': 200, 'stepsize': 4}
20 200 4
Pulling In Sample Data!!
Generating Moving Average Ribbon Signal!!
Generating Volatility Adjusted Trend Signal!!
Getting Average True Range for Stop Loss Calcul

In [691]:
df_performance_stepsize_4.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Stepsize-2023-10-01-2025-10-01.pickle')

In [692]:
df_performance_stepsize = pd.concat([df_performance_stepsize_1, df_performance_stepsize_2, df_performance_stepsize_3, df_performance_stepsize_4], axis=0, ignore_index=True)

In [703]:
df_performance_stepsize.to_pickle('/Users/adheerchauhan/Documents/git/trend_following/trend_following_results/Portfolio_Moving_Average_Ribbon_Performance_Stepsize-2022-04-01-2025-10-01.pickle')

In [705]:
df_performance_stepsize_is = df_performance_stepsize[df_performance_stepsize.sampling_category == 'in_sample']
df_performance_stepsize_os = df_performance_stepsize[df_performance_stepsize.sampling_category != 'in_sample']

In [707]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_stepsize_is.groupby(['fast_mavg','slow_mavg','mavg_stepsize']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                                   median      mean       std   
fast_mavg slow_mavg mavg_stepsize                                               
20        200       2                            1.198108  0.876979  0.706840   
                    4                            1.196296  0.896957  0.682397   
                    6                            1.204964  0.911992  0.682902   
                    8                            1.204199  0.916628  0.685358   
                    10                           1.196210  0.912972  0.685443   
                    12                           1.196446  0.915258  0.686809   
                    14                           1.197402  0.916722  0.687968   

                                  annualized_return                      \
                                             median      mean       std   
fast_mavg slow_mavg mavg_stepsize                                         
20        200       2                      0.631723  0.469819  0.419115   
                    4                      0.630508  0.480978  0.412884   
                    6                      0.637410  0.492383  0.417944   
                    8                      0.636918  0.496599  0.421799   
                    10                     0.630839  0.494264  0.422135   
                    12                     0.630947  0.496316  0.424039   
                    14                     0.631895  0.497672  0.425233   

                                  max_drawdown                      \
                                        median      mean       std   
fast_mavg slow_mavg mavg_stepsize                                    
20        200       2                -0.303012 -0.354299  0.169486   
                    4                -0.298919 -0.352624  0.166900   
                    6                -0.295461 -0.350448  0.163365   
                    8                -0.294313 -0.349551  0.162287   
                    10               -0.293059 -0.348881  0.161462   
                    12               -0.292500 -0.348559  0.161349   
                    14               -0.292049 -0.348319  0.161162   

                                  BTC-USD_annualized_sharpe_ratio            \
                                                           median      mean   
fast_mavg slow_mavg mavg_stepsize                                             
20        200       2                                    0.188047  0.284547   
                    4                                    0.187986  0.293303   
                    6                                    0.201400  0.293264   
                    8                                    0.203160  0.292045   
                    10                                   0.198573  0.290179   
                    12                                   0.200327  0.289370   
                    14                                   0.202341  0.289966   

                                            ETH-USD_annualized_sharpe_ratio  \
                                        std                          median   
fast_mavg slow_mavg mavg_stepsize                                             
20        200       2              0.580447                        0.282465   
                    4              0.587842                        0.316380   
                    6              0.596828                        0.349278   
                    8              0.597792                        0.363187   
                    10             0.599732                        0.369666   
                    12             0.605571                        0.377428   
                    14             0.603991                        0.381014   

                                                       \
                                       mean       std   
fast_mavg slow_mavg mavg_stepsize                       
20        200     

In [709]:
agg_dict = {'annualized_sharpe_ratio':['median','mean','std'],
            'annualized_return':['median','mean','std'],
            'max_drawdown':['median','mean','std'],
            'BTC-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ETH-USD_annualized_sharpe_ratio':['median','mean','std'],
            'SOL-USD_annualized_sharpe_ratio':['median','mean','std'],
            'ADA-USD_annualized_sharpe_ratio':['median','mean','std'],
            'AVAX-USD_annualized_sharpe_ratio':['median','mean','std']}
df_performance_stepsize_os.groupby(['fast_mavg','slow_mavg','mavg_stepsize']).agg(agg_dict)

annualized_sharpe_ratio                      \
                                                   median      mean       std   
fast_mavg slow_mavg mavg_stepsize                                               
20        200       4                            0.097027  0.097027       NaN   
                    8                            1.938065  1.938065  1.072279   
                    14                          -1.357361 -1.357361       NaN   

                                  annualized_return                      \
                                             median      mean       std   
fast_mavg slow_mavg mavg_stepsize                                         
20        200       4                     -0.024184 -0.024184       NaN   
                    8                      1.638222  1.638222  1.356337   
                    14                    -0.479793 -0.479793       NaN   

                                  max_drawdown                      \
                                        median      mean       std   
fast_mavg slow_mavg mavg_stepsize                                    
20        200       4                -0.208694 -0.208694       NaN   
                    8                -0.256967 -0.256967  0.065631   
                    14               -0.280546 -0.280546       NaN   

                                  BTC-USD_annualized_sharpe_ratio            \
                                                           median      mean   
fast_mavg slow_mavg mavg_stepsize                                             
20        200       4                                   -1.247564 -1.247564   
                    8                                    1.159483  1.159483   
                    14                                  -0.508331 -0.508331   

                                            ETH-USD_annualized_sharpe_ratio  \
                                        std                          median   
fast_mavg slow_mavg mavg_stepsize                                             
20        200       4                   NaN                       -0.758636   
                    8              0.055011                        0.763285   
                    14                  NaN                       -0.580481   

                                                       \
                                       mean       std   
fast_mavg slow_mavg mavg_stepsize                       
20        200       4             -0.758636       NaN   
                    8              0.763285  0.807716   
                    14            -0.580481       NaN   

                                  SOL-USD_annualized_sharpe_ratio            \
                                                           median      mean   
fast_mavg slow_mavg mavg_stepsize                                             
20        200       4                                   -0.837243 -0.837243   
                    8                                    0.828266  0.828266   
                    14                                  -0.957390 -0.957390   

                                            ADA-USD_annualized_sharpe_ratio  \
                                        std                          median   
fast_mavg slow_mavg mavg_stepsize                                             
20        200       4                   NaN                        0.903388   
                    8              2.970098                        1.455656   
                    14                  NaN                       -1.098140   

                                                       \
                                       mean       std   
fast_mavg slow_mavg mavg_stepsize                       
20        200       4              0.903388       NaN   
                    8              1.455656  0.869636   
                    14            -1.098140       NaN   

                                  AVAX-USD_annualized_sharpe_ratio            \
         